In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import os
os.chdir(r"C:\Users\PC Jules\Desktop\EcoDynElec_UI\ecodynelec")

Loading data from the data created by the swiss_mix_Souh

In [2]:
# Données flows
flows_FR = pd.read_parquet("./data/flows/flows_FR.parquet.gz")
flows_DE = pd.read_parquet("./data/flows/flows_DE.parquet.gz")
flows_AT = pd.read_parquet("./data/flows/flows_AT.parquet.gz")
flows_CH = pd.read_parquet("./data/flows/flows_CH.parquet.gz")
flows_IT = pd.read_parquet("./data/flows/flows_IT.parquet.gz")

# Données de consommation totale
tot_consumption_FR = pd.read_parquet("./data/consumptions/tot_consumption_FR.parquet.gz")
tot_consumption_DE = pd.read_parquet("./data/consumptions/tot_consumption_DE.parquet.gz")
tot_consumption_AT = pd.read_parquet("./data/consumptions/tot_consumption_AT.parquet.gz")
tot_consumption_CH = pd.read_parquet("./data/consumptions/tot_consumption_CH.parquet.gz")
tot_consumption_IT = pd.read_parquet("./data/consumptions/tot_consumption_IT.parquet.gz")

# Données de consommation by src
raw_consumption_by_src_FR = pd.read_parquet("./data/consumptions/raw_consumption_by_src_FR.parquet.gz")
raw_consumption_by_src_DE = pd.read_parquet("./data/consumptions/raw_consumption_by_src_DE.parquet.gz")
raw_consumption_by_src_AT = pd.read_parquet("./data/consumptions/raw_consumption_by_src_AT.parquet.gz")
raw_consumption_by_src_CH = pd.read_parquet("./data/consumptions/raw_consumption_by_src_CH.parquet.gz")
raw_consumption_by_src_IT = pd.read_parquet("./data/consumptions/raw_consumption_by_src_IT.parquet.gz")

# electricity_mixs
tot_electricity_mix_CH = pd.read_parquet("./data/electricity_mixs/electricity_mix_CH.parquet.gz")
tot_electricity_mix_AT = pd.read_parquet("./data/electricity_mixs/electricity_mix_AT.parquet.gz")
tot_electricity_mix_DE = pd.read_parquet("./data/electricity_mixs/electricity_mix_DE.parquet.gz")
tot_electricity_mix_FR = pd.read_parquet("./data/electricity_mixs/electricity_mix_FR.parquet.gz")
tot_electricity_mix_IT = pd.read_parquet("./data/electricity_mixs/electricity_mix_IT.parquet.gz")

# electricity_impacts
tot_electricity_impact_CH = pd.read_parquet("./data/electricity_impacts/electricity_impact_CH.parquet.gz")
tot_electricity_impact_AT = pd.read_parquet("./data/electricity_impacts/electricity_impact_AT.parquet.gz")
tot_electricity_impact_DE = pd.read_parquet("./data/electricity_impacts/electricity_impact_DE.parquet.gz")
tot_electricity_impact_FR = pd.read_parquet("./data/electricity_impacts/electricity_impact_FR.parquet.gz")
tot_electricity_impact_IT = pd.read_parquet("./data/electricity_impacts/electricity_impact_IT.parquet.gz")

# electricity_impacts by source
electricity_impact_by_src_CH = pd.read_parquet("./data/electricity_impacts/electricity_impact_by_src_CH.parquet.gz")
electricity_impact_by_src_AT = pd.read_parquet("./data/electricity_impacts/electricity_impact_by_src_AT.parquet.gz")
electricity_impact_by_src_DE = pd.read_parquet("./data/electricity_impacts/electricity_impact_by_src_DE.parquet.gz")
electricity_impact_by_src_FR = pd.read_parquet("./data/electricity_impacts/electricity_impact_by_src_FR.parquet.gz")
electricity_impact_by_src_IT = pd.read_parquet("./data/electricity_impacts/electricity_impact_by_src_IT.parquet.gz")

# Technologies
Techno_FR = pd.read_parquet("./data/technologies/technologies_FR.parquet.gz")
Techno_AT = pd.read_parquet("./data/technologies/technologies_AT.parquet.gz")
Techno_DE = pd.read_parquet("./data/technologies/technologies_DE.parquet.gz")
Techno_CH = pd.read_parquet("./data/technologies/technologies_CH.parquet.gz")
Techno_IT = pd.read_parquet("./data/technologies/technologies_IT.parquet.gz")

# Technologies impact
Techno_impact_FR = pd.read_parquet("./data/technologies/Techno_impact_FR.parquet.gz")
Techno_impact_AT = pd.read_parquet("./data/technologies/Techno_impact_AT.parquet.gz")
Techno_impact_DE = pd.read_parquet("./data/technologies/Techno_impact_DE.parquet.gz")
Techno_impact_CH = pd.read_parquet("./data/technologies/Techno_impact_CH.parquet.gz")
Techno_impact_IT = pd.read_parquet("./data/technologies/Techno_impact_IT.parquet.gz")

for df in [flows_FR, flows_DE, flows_AT, flows_CH, tot_consumption_FR, tot_consumption_DE, tot_consumption_AT, tot_consumption_CH,
           tot_electricity_mix_CH,tot_electricity_mix_AT,tot_electricity_mix_DE,tot_electricity_mix_FR,tot_electricity_impact_CH,
           tot_electricity_impact_AT,tot_electricity_impact_DE,tot_electricity_impact_FR,raw_consumption_by_src_FR,raw_consumption_by_src_CH,
           raw_consumption_by_src_DE,raw_consumption_by_src_AT,electricity_impact_by_src_CH,electricity_impact_by_src_AT,
           electricity_impact_by_src_DE,electricity_impact_by_src_FR,Techno_FR,Techno_AT,Techno_DE,Techno_CH,Techno_impact_FR,Techno_impact_AT,
            Techno_impact_DE,Techno_impact_CH,flows_IT,tot_consumption_IT,raw_consumption_by_src_IT,tot_electricity_mix_IT,
           tot_electricity_impact_IT,electricity_impact_by_src_IT,Techno_IT,Techno_impact_IT]:

        # Only rename if 'Unnamed: 0' column exists
        if 'Unnamed: 0' in df.columns:
            df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)
        else:
            # For parquet files, the index is already set, just ensure it's datetime
            if not isinstance(df.index, pd.DatetimeIndex):
                df.index = pd.to_datetime(df.index)

for df in [flows_FR, flows_DE, flows_AT, flows_CH,flows_IT]:
    df['total_consumption']=df['production']+df['imports']-df['exports']

# Dictionary of dataframes with Italy added
dataframes_flows = {'Switzerland': flows_CH,
    'France': flows_FR,
    'Germany': flows_DE,
    'Austria': flows_AT,
    'Italy': flows_IT
}

dataframes_tot_consumption = {
    'Switzerland': tot_consumption_CH,
    'France': tot_consumption_FR,
    'Germany': tot_consumption_DE,
    'Austria': tot_consumption_AT,
    'Italy': tot_consumption_IT
}

dataframes_raw_consumption_by_src = {
    'Switzerland': raw_consumption_by_src_CH,
    'France': raw_consumption_by_src_FR,
    'Germany': raw_consumption_by_src_DE,
    'Austria': raw_consumption_by_src_AT,
    'Italy': raw_consumption_by_src_IT
}

dataframes_tot_electricity_mix = {
    'Switzerland': tot_electricity_mix_CH,
    'France': tot_electricity_mix_FR,
    'Germany': tot_electricity_mix_DE,
    'Austria': tot_electricity_mix_AT,
    'Italy': tot_electricity_mix_IT
}

dataframes_tot_electricity_impact = {
    'Switzerland': tot_electricity_impact_CH,
    'France': tot_electricity_impact_FR,
    'Germany': tot_electricity_impact_DE,
    'Austria': tot_electricity_impact_AT,
    'Italy': tot_electricity_impact_IT
}

dataframes_electricity_impact_by_src = {
    'Switzerland': electricity_impact_by_src_CH,
    'France': electricity_impact_by_src_FR,
    'Germany': electricity_impact_by_src_DE,
    'Austria': electricity_impact_by_src_AT,
    'Italy': electricity_impact_by_src_IT
}

dataframes_techno = {
    'Switzerland': Techno_CH,
    'France': Techno_FR,
    'Germany': Techno_DE,
    'Austria': Techno_AT,
    'Italy': Techno_IT
}

dataframes_techno_impact = {
    'Switzerland': Techno_impact_CH,
    'France': Techno_impact_FR,
    'Germany': Techno_impact_DE,
    'Austria': Techno_impact_AT,
    'Italy': Techno_impact_IT
}

os.chdir(r"C:\Users\PC Jules\Desktop\SwissMeteo_for_Energy\data")

meteo = pd.read_parquet('meteo_data.parquet.gz')
meteo_data = meteo.copy()

## I. Temporal analysis
### I.1. Annual view
#### a) Temporal evolution of consumption

In [3]:
# Prepare meteo data : Calculating HDD
meteo_data = meteo_data.resample('D').mean()
T_base = 18
meteo_data['HDD'] = (T_base - meteo_data['air_temperature_global']).clip(lower=0)
HDD_monthly = meteo_data['HDD'].resample('ME').sum()
HDD_year =  meteo_data['HDD'].resample('YE').sum()

In [4]:
# Color for each years
color = {
    # --- Pays & Imports ---
    'CH': '#026849',
    'FR': '#9EDF8A',
    'DE': '#42A678',
    'IT': '#54DFFE',
    'AT': '#E4E88F',
    'Other': '#D8B59E',

    # --- Saisons (Pour les Boxplots) ---
    'Winter': '#00502A',
    'Summer': '#FF8C00',

    # --- Bases de Données (Pour le comparatif ACV) ---
    'Ecoinvent': '#636EFA',                   # Bleu
    'Horocarbon': '#00CC96',                  # Vert
    'UVEK': '#EF553B',                        # Rouge
    'ElectricityMaps': '#AB63FA',             # Violet

    # --- Années (Comparaison historique) ---
    '2018': '#DCDCDC',
    '2019': '#C0C0C0',
    '2020': '#A0A0A0',
    '2021': '#808080',
    '2022': '#696969',
    '2023': '#404040',                        # Gris très sombre
    '2024': '#FF0000'                         # Rouge (Année de rupture)
}

# Prepare data for Switzerland from 2016 to 2024
df_ch = tot_consumption_CH.loc['2016':'2024'].copy() / 1000 # Convert to GWh

# ===== DAILY DATA =====
df_ch_daily = df_ch.resample('D').sum()

# ===== MONTHLY DATA =====
df_ch_monthly = df_ch.resample('ME').sum()

# ===== COMMON PARAMETERS =====
years_to_plot = [2018, 2019, 2020, 2021, 2022, 2023, 2024]

# ===== GRAPH 1: DAILY AVERAGES =====
fig = go.Figure()

fig.update_xaxes(
    showgrid=True,
    gridcolor="lightgray",
    gridwidth=0.8
)

for i, year in enumerate(years_to_plot):
    df = df_ch_daily.loc[str(year)].copy()

    fig.add_trace(go.Scatter(
        x=df.index.month_name(),
        y=df['sum'],
        name=str(year),
        line=dict(width=1, color=color[str(year)]),
        mode='lines'
    ))

fig.update_xaxes(
    title=dict(font=dict(size=12, color="black")),
    tickfont=dict(size=10, color="black"),
    tickformat="%B",
    dtick="M1",
    showline=True,
    linewidth=0.8,
    mirror=True,
    linecolor='lightgray',
    zeroline=True,
    zerolinecolor='lightgray',
    zerolinewidth=0.8,
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
)

fig.update_yaxes(
    range=[100,240],
    dtick=28,
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showline=True,
    linewidth=0.8,
    mirror=True,
    linecolor='lightgray',
    zeroline=True,
    zerolinecolor='lightgray',
    zerolinewidth=0.8,
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
)

fig.update_layout(
    title=dict(text="Swiss Electricity Consumption (2016-2024) - Daily sum",x=0.5, font=dict(size=14, color="black", family="Arial")),
    yaxis_title="Consumption [GWh]",
    xaxis_title="Month",
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.35,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    font=dict(
        size=9,
        color="black",
        family = 'Arial'
    )
)

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\daily_avg_consumption.png",width=591,
                height=333, scale=3)

fig.show()

# ===== GRAPH 2: MONTHLY AVERAGES =====
fig = go.Figure()

fig.update_xaxes(
    showgrid=True,
    gridcolor="lightgray",
    gridwidth=0.8
)

for i, year in enumerate(years_to_plot):
    df = df_ch_monthly.loc[str(year)]
    fig.add_trace(go.Scatter(
        x=df.index.month_name(), y=df['sum'], name=str(year),
        line=dict(width=1, color=color[str(year)]),
    ))

fig.update_xaxes(
    title=dict(font=dict(size=12, color="black")),
    tickfont=dict(size=10, color="black"),
    tickformat="%B",
    dtick="M1",
    showline=True,
    linewidth=0.8,
    mirror=True,
    linecolor='lightgray',
    zeroline=True,
    zerolinecolor='lightgray',
    zerolinewidth=0.8,
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
)

fig.update_yaxes(
    range=[4500,6500],
    dtick=500,
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showline=True,
    linewidth=0.8,
    mirror=True,
    linecolor='lightgray',
    zeroline=True,
    zerolinecolor='lightgray',
    zerolinewidth=0.8,
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
)

fig.update_layout(
    title=dict(text="Swiss Electricity Consumption (2016-2024) - Monthly Sum",x=0.5, font=dict(size=14, color="black", family="Arial")),
    yaxis_title="Consumption [GWh]",
    xaxis_title="Month",
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.35,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    margin=dict(l=60, r=60, t=60, b=100),
    font=dict(
        size=9,
        color="black",
        family = 'Arial'
    )
)

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\monthly_avg_consumption.png",width=591,
                height=333, scale=3)

fig.show()

# ===== GRAPH 3: HDD vs Consumption + GRAPH 4 : HDD Regression =====
df_ch_monthly = df_ch.resample('ME').sum()
fig1 = go.Figure()

fig1.update_xaxes(
    showgrid=True,
    gridcolor="lightgray",
    gridwidth=0.8
)

titles_list = [str(year) for year in years_to_plot[:-1]]
titles_list.extend(["", str(years_to_plot[-1]), ""])

fig = make_subplots(
    rows=3, cols=3,
    subplot_titles=titles_list,
    vertical_spacing=0.10,
    horizontal_spacing=0.08
)
fig.update_annotations(font_size=10)

for i, year in enumerate(years_to_plot):
    row = i // 3 + 1
    col = i % 3 + 1

    for y in years_to_plot:
        if year == 2024:
            row = 3
            col = 2
        df_background = df_ch_monthly.loc[str(y)]
        fig.add_trace(
            go.Scatter(
                x=HDD_monthly.loc[str(y)].values,
                y=df_background['sum'].values,
                mode='markers',
                marker=dict(
                    color='lightgray',
                    size=6,
                    opacity=0.4
                ),
                showlegend=False,
                hovertemplate='HDD: %{x:.0f}<br>Consumption: %{y:.1f} GWh<extra></extra>'
            ),
            row=row, col=col
        )

    df = df_ch_monthly.loc[str(year)]
    x = HDD_monthly.loc[str(year)].values
    y = df['sum'].values

    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode='markers',
            marker=dict(
                color='darkred',
                size=10,
                opacity=0.8,
                line=dict(color='black', width=0.5)
            ),
            name=f'{year}',
            showlegend=False,
            hovertemplate='HDD: %{x:.0f}<br>Consumption: %{y:.1f} GWh<extra></extra>'
        ),
        row=row, col=col
    )

    if len(x) > 1:
        slope, intercept = np.polyfit(x, y, 1)
        x_sorted = np.sort(x)
        y_pred = intercept + slope * x_sorted

        fig.add_trace(
            go.Scatter(
                x=x_sorted,
                y=y_pred,
                mode='lines',
                line=dict(
                    color='black',
                    width=1,
                    dash='dash'
                ),
                name=f'y = {slope:.2f}x + {intercept:.0f}',
                showlegend=False,
                hovertemplate='Regression<extra></extra>'
            ),
            row=row, col=col
        )

        fig1.add_trace(
            go.Scatter(
                x=x_sorted,
                y=y_pred,
                mode='lines',
                line=dict(
                    color=color[str(year)],
                    width=1
                ),
                name=f'{year} : y = {slope:.2f}x + {intercept:.0f}',
                showlegend=False,
                hovertemplate='Regression<extra></extra>'
            )
        )
        

        fig.add_annotation(
            x=0.05,
            y=0.95,
            xref=f'x{i+1} domain' if i > 0 else 'x domain',
            yref=f'y{i+1} domain' if i > 0 else 'y domain',
            text=f'<span style="color:darkred;">●</span> <b>{year}</b><br>--- y = {slope:.2f}x + {intercept:.0f}',
            showarrow=False,
            font=dict(size=8, color='black', family='Arial'),
            align='left',
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='lightgray',
            borderwidth=1,
            borderpad=4
        )

    fig.update_xaxes(
        title_text='HDD',
        range=[0,700],
        dtick=175,
        title=dict(font=dict(size=12, color="black")),
        tickfont=dict(size=10, color="black"),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=row, col=col
        )

    if i in [0,3,6]:
        fig.update_yaxes(
            title_text='Consumption (GWh)',
            title_font=dict(size=12, color='black', family='Arial'),
            tickfont=dict(size=10, color='black', family='Arial'),
            showline=True,
            linewidth=0.8,
            mirror=True,
            linecolor='lightgray',
            zeroline=True,
            zerolinecolor='lightgray',
            zerolinewidth=0.8,
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=0.8,
            row=row, col=col
        )

fig.update_yaxes(
        range=[4400,6400],
        dtick=400,
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        )

fig1.update_xaxes(
        title_text='HDD',
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
    )

fig1.update_yaxes(
        title_text='Consumption (GWh)',
        range=[4400,6400],
        dtick=400,
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        )

fig.update_layout(
    title=dict(
        text='Evolution of Swiss electricity consumption vs HDD (2018-2024)',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    height=1200,
    width=1400,
    template='plotly_white',
    showlegend=False,
    font=dict(
        size=9,
        color='black',
        family='Arial'
    ),
    hovermode='closest'
)

fig1.update_layout(
    title=dict(
        text='Evolution of Swiss electricity consumption vs HDD - Regression (2018-2024) ',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    template='plotly_white',
    showlegend=False,
    font=dict(
        size=9,
        color='black',
        family='Arial'
    ),
    hovermode='closest'
)

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\consumption_vs_hdd_subplots.png",
                width=591, height=750, scale=3)

fig1.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\consumption_vs_hdd_regression.png",
                width=591, height=333, scale=3)

fig.show()
fig1.show()

# ===== STATISTICAL SUMMARY =====
print("=" * 80)
print("SWISS ELECTRICITY CONSUMPTION (2016-2024) - STATISTICAL SUMMARY")
print("=" * 80)

df_ch['year'] = df_ch.index.year
df_ch['day_of_year'] = df_ch.index.dayofyear

# Calculate annual statistics
print("\n--- ANNUAL CONSUMPTION (GWh) ---")
annual_total = df_ch.groupby('year')['sum'].sum() / 1000  # Convert to GWh
annual_mean = df_ch.groupby('year')['sum'].mean()  # Keep in MWh for hourly average

for year in years_to_plot:
    if year in annual_total.index:
        print(f"{year}: Total = {annual_total[year]:,.2f} GWh | "
              f"Hourly Average = {annual_mean[year]:,.2f} MWh")

# Overall trend
total_2016 = annual_total[2016] if 2016 in annual_total.index else None
total_2024 = annual_total[2024] if 2024 in annual_total.index else None
if total_2016 and total_2024:
    change = ((total_2024 - total_2016) / total_2016) * 100
    print(f"\nTotal Change (2016 → 2024): {change:+.2f}%")

# Daily average statistics by year
df_ch_daily = df_ch.groupby(['year', 'day_of_year']).mean()
df_ch_daily = df_ch_daily.reset_index()

print("\n--- DAILY AVERAGE CONSUMPTION (GWh) ---")
daily_stats = df_ch_daily.groupby('year')['sum'].agg(['mean', 'std', 'min', 'max'])
print(f"{'Year':<8} {'Mean':<12} {'Std Dev':<12} {'Min':<12} {'Max':<12}")
print("-" * 60)
for year in years_to_plot:
    if year in daily_stats.index:
        stats = daily_stats.loc[year]
        print(f"{year:<8} {stats['mean']:>10,.2f}   {stats['std']:>10,.2f}   "
              f"{stats['min']:>10,.2f}   {stats['max']:>10,.2f}")

# Monthly average statistics by year
df_ch['month'] = df_ch.index.month

# Group by year and month, then calculate monthly average
df_ch_monthly = df_ch.groupby(['year', 'month']).mean()
df_ch_monthly = df_ch_monthly.reset_index()

print("\n--- MONTHLY AVERAGE CONSUMPTION (GWh) ---")
monthly_stats = df_ch_monthly.groupby('year')['sum'].agg(['mean', 'std', 'min', 'max'])
print(f"{'Year':<8} {'Mean':<12} {'Std Dev':<12} {'Min':<12} {'Max':<12}")
print("-" * 60)
for year in years_to_plot:
    if year in monthly_stats.index:
        stats = monthly_stats.loc[year]
        print(f"{year:<8} {stats['mean']:>10,.2f}   {stats['std']:>10,.2f}   "
              f"{stats['min']:>10,.2f}   {stats['max']:>10,.2f}")

# Seasonal analysis (for all years combined)
print("\n--- SEASONAL PATTERNS (All Years Combined) ---")
season_mapping = {12: 'Winter', 1: 'Winter', 2: 'Winter', 3: 'Spring',
                  4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer',
                  8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall'}
df_ch['season'] = df_ch['month'].map(season_mapping)
seasonal_avg = df_ch.groupby('season')['sum'].mean()

season_order = ['Winter', 'Spring', 'Summer', 'Fall']
print(f"{'Season':<12} {'Average Consumption (GWh)':<25}")
print("-" * 40)
for season in season_order:
    if season in seasonal_avg.index:
        print(f"{season:<12} {seasonal_avg[season]:>20,.2f}")

# Peak consumption analysis
print("\n--- PEAK CONSUMPTION ANALYSIS ---")
for year in years_to_plot:
    year_data = df_ch_daily[df_ch_daily['year'] == year]
    if not year_data.empty:
        max_val = year_data['sum'].max()
        max_date = year_data.loc[year_data['sum'].idxmax(), 'day_of_year']
        # Convert day_of_year to approximate date
        approx_date = pd.Timestamp(year=year, month=1, day=1) + pd.Timedelta(days=int(max_date)-1)
        print(f"{year}: {max_val:,.2f} GWh on day {int(max_date)} "
              f"(~{approx_date.strftime('%b %d')})")

# Year-over-year growth rates
print("\n--- YEAR-OVER-YEAR GROWTH RATES ---")
for i in range(len(years_to_plot)-1):
    year1, year2 = years_to_plot[i], years_to_plot[i+1]
    if year1 in annual_total.index and year2 in annual_total.index:
        growth = ((annual_total[year2] - annual_total[year1]) / annual_total[year1]) * 100
        print(f"{year1} → {year2}: {growth:+.2f}%")

# Coefficient of variation (volatility measure)
print("\n--- CONSUMPTION VOLATILITY (Coefficient of Variation) ---")
for year in years_to_plot:
    year_data = df_ch_daily[df_ch_daily['year'] == year]
    if not year_data.empty:
        cv = (year_data['sum'].std() / year_data['sum'].mean()) * 100
        print(f"{year}: {cv:.2f}% (lower = more stable)")

print("\n--- HDD–CONSUMPTION LINEAR REGRESSION ANALYSIS ---")
print(f"{'Year':<8} {'Slope':<12} {'Intercept':<12} {'R² Score':<10} {'Interpretation'}")

slope_trends = []

for year in years_to_plot:
    # Prepare data for each year
    data_year = df_ch_monthly[df_ch_monthly['year'] == year]
    if str(year) not in HDD_monthly.index:
        continue

    X = HDD_monthly.loc[str(year)].values.reshape(-1, 1)
    y = data_year['sum'].values

    # Check for valid data
    if len(X) < 2 or len(y) < 2:
        continue

    # Linear regression
    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)

    slope = model.coef_[0]
    intercept = model.intercept_
    r2 = r2_score(y, y_pred)
    slope_trends.append(slope)

    # Qualitative interpretation
    if r2 >= 0.8:
        quality = "very strong correlation"
    elif r2 >= 0.6:
        quality = "moderate correlation"
    elif r2 >= 0.4:
        quality = "weak correlation"
    else:
        quality = "no significant correlation"

    trend = "positive" if slope > 0 else "negative"
    print(f"{year:<8} {slope:>+10.2f}   {intercept:>10.2f}   {r2:>8.3f}   "
          f"{trend.capitalize()} slope, {quality}")

# ---- Global Trend Summary ----
print("\n--- SUMMARY INTERPRETATION ---")

mean_slope = np.mean(slope_trends) if slope_trends else np.nan
mean_slope_str = f"{mean_slope:+.2f}" if not np.isnan(mean_slope) else "N/A"

if mean_slope > 0:
    direction = "increasing"
    implication = (
        "indicating that higher HDD (colder months) are generally associated "
        "with higher electricity consumption, likely due to heating demand."
    )
else:
    direction = "decreasing"
    implication = (
        "suggesting that colder months tend to show lower or unchanged electricity use, "
        "possibly due to improvements in building insulation or a shift to non-electric heating."
    )

print(f"Average slope across all years: {mean_slope_str}")
print(f"Overall trend: {direction} relationship between HDD and consumption, {implication}")
print("=" * 80)

SWISS ELECTRICITY CONSUMPTION (2016-2024) - STATISTICAL SUMMARY

--- ANNUAL CONSUMPTION (GWh) ---
2018: Total = 63.66 GWh | Hourly Average = 7.27 MWh
2019: Total = 63.48 GWh | Hourly Average = 7.25 MWh
2020: Total = 62.45 GWh | Hourly Average = 7.11 MWh
2021: Total = 64.46 GWh | Hourly Average = 7.36 MWh
2022: Total = 64.72 GWh | Hourly Average = 7.39 MWh
2023: Total = 62.10 GWh | Hourly Average = 7.09 MWh
2024: Total = 61.66 GWh | Hourly Average = 7.02 MWh

Total Change (2016 → 2024): -2.70%

--- DAILY AVERAGE CONSUMPTION (GWh) ---
Year     Mean         Std Dev      Min          Max         
------------------------------------------------------------
2018           7.27         0.85         5.42         9.53
2019           7.25         0.78         4.99         8.96
2020           7.11         0.75         5.51         8.93
2021           7.36         0.85         5.67         9.39
2022           7.39         0.80         5.45         9.38
2023           7.09         0.83         5.3

In [5]:
# Prepare GHG emissions data for Switzerland from 2016 to 2024
df_ghg = tot_electricity_impact_CH.loc['2016':'2024'].copy()

# Create columns for year and day of year
df_ghg['year'] = df_ghg.index.year
df_ghg['day_of_year'] = df_ghg.index.dayofyear

# ===== DAILY DATA =====x
df_ghg_daily = df_ghg.resample('D').mean()

# ===== MONTHLY DATA =====
df_ghg_monthly = df_ghg.resample('ME').mean()

# ===== COMMON PARAMETERS =====
years_to_plot = [2018, 2019, 2020, 2021, 2022, 2023, 2024]

# ===== GRAPH 1: BOXPLOT =====
fig = go.Figure()

for i, year in enumerate(years_to_plot):
    df = df_ghg_daily.loc[str(year)].copy()

    fig.add_trace(
        go.Box(
            y=df['sum'],
            name=str(year),
            line=dict(width=1,color=color[str(year)]),
            marker=dict(color=color[str(year)], opacity=0.8),
            boxmean=True,
            showlegend=False
        )
    )

fig.update_xaxes(
    title=dict(font=dict(size=12, color="black")),
    tickfont=dict(size=10, color="black"),
    tickformat="%B",
    dtick="M1",
    showline=True,
    linewidth=0.8,
    mirror=True,
    linecolor='lightgray',
    zeroline=True,
    zerolinecolor='lightgray',
    zerolinewidth=0.8,
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
)

fig.update_yaxes(
    range=[0,300],
    dtick=50,
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showline=True,
    linewidth=0.8,
    mirror=True,
    linecolor='lightgray',
    zeroline=True,
    zerolinecolor='lightgray',
    zerolinewidth=0.8,
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
)

fig.update_layout(
    title=dict(text="Swiss Electricity yearly GHG Emissions boxplot (2016-2024)",x=0.5, font=dict(size=14, color="black", family="Arial")),
    yaxis_title="GHG emissions [gCO2eq/kWh]",
    xaxis_title="Year",
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.35,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    yaxis=dict(rangemode='tozero'),
    font=dict(
        size=9,
        color="black",
        family = 'Arial'
    )
)

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\yearly_GHG_boxplot.png",width=591,
                height=333, scale=3)

fig.show()

# ===== GRAPH 3: MONTHLY AVERAGES =====
fig = go.Figure()

fig.update_xaxes(
    showgrid=True,
    gridcolor="lightgray",
    gridwidth=0.8
)

for i, year in enumerate(years_to_plot):
    df = df_ghg_monthly.loc[str(year)]
    fig.add_trace(go.Scatter(
        x=df.index.month_name(), y=df['sum'], name=str(year),
        line=dict(width=1, color=color[str(year)]),
    ))

fig.update_xaxes(
    title=dict(font=dict(size=12, color="black")),
    tickfont=dict(size=10, color="black"),
    showline=True,
    linewidth=0.8,
    mirror=True,
    linecolor='lightgray',
    zeroline=True,
    zerolinecolor='lightgray',
    zerolinewidth=0.8,
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
)

fig.update_yaxes(
    range=[0,250],
    dtick=50,
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showline=True,
    linewidth=0.8,
    mirror=True,
    linecolor='lightgray',
    zeroline=True,
    zerolinecolor='lightgray',
    zerolinewidth=0.8,
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
)

fig.update_layout(
    title=dict(text="Swiss Electricity GHG Emissions (2016-2024) - Monthly Average",x=0.5, font=dict(size=14, color="black", family="Arial")),
    yaxis_title="GHG emissions [gCO2eq/kWh]",
    xaxis_title="Month",
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.4,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    yaxis=dict(rangemode='tozero'),
    margin=dict(l=60, r=60, t=60, b=100),
    font=dict(
        size=9,
        color="black",
        family = 'Arial'
    )
)

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\monthly_avg_GHG.png",width=591,
                height=333, scale=3)

fig.show()

# ===== STATISTICAL SUMMARY =====
print("=" * 80)
print("SWISS CONSUMPTION MIX GHG EMISSIONS (2016-2024) - STATISTICAL SUMMARY")
print("=" * 80)

# Calculate annual statistics
df_ghg['year'] = df_ghg.index.year
df_ghg['day_of_year'] = df_ghg.index.dayofyear

print("\n--- ANNUAL AVERAGE EMISSIONS INTENSITY (gCO2eq/kWh) ---")
annual_median = df_ghg.groupby('year')['sum'].median()
annual_mean = df_ghg.groupby('year')['sum'].mean()
for year in years_to_plot:
    if year in annual_mean.index:
        print(f"{year}: Mean = {annual_mean[year]:.2f} gCO2eq/kWh | "
              f"Median = {annual_median[year]:.2f} gCO2eq/kWh")

# Overall trend
emissions_2016 = annual_mean[2016] if 2016 in annual_mean.index else None
emissions_2024 = annual_mean[2024] if 2024 in annual_mean.index else None
if emissions_2016 and emissions_2024:
    change = ((emissions_2024 - emissions_2016) / emissions_2016) * 100
    absolute_change = emissions_2024 - emissions_2016
    print(f"\nChange (2016 → 2024): {change:+.2f}% ({absolute_change:+.2f} gCO2eq/kWh)")

# Daily average statistics by year
df_ghg_daily = df_ghg.groupby(['year', 'day_of_year']).mean()
df_ghg_daily = df_ghg_daily.reset_index()

print("\n--- DAILY AVERAGE EMISSIONS (gCO2eq/kWh) ---")
daily_stats = df_ghg_daily.groupby('year')['sum'].agg(['mean', 'std', 'min', 'max'])
print(f"{'Year':<8} {'Mean':<12} {'Std Dev':<12} {'Min':<12} {'Max':<12}")
print("-" * 60)
for year in years_to_plot:
    if year in daily_stats.index:
        stats = daily_stats.loc[year]
        print(f"{year:<8} {stats['mean']:>10.2f}   {stats['std']:>10.2f}   "
              f"{stats['min']:>10.2f}   {stats['max']:>10.2f}")

# Monthly average statistics by year
# Monthly average statistics by year
df_ghg['month'] = df_ghg.index.month

# Group by year and month, then calculate monthly average
df_ghg_monthly = df_ghg.groupby(['year', 'month']).mean()
df_ghg_monthly = df_ghg_monthly.reset_index()

print("\n--- MONTHLY AVERAGE EMISSIONS (gCO2eq/kWh) ---")
monthly_stats = df_ghg_monthly.groupby('year')['sum'].agg(['mean', 'std', 'min', 'max'])
print(f"{'Year':<8} {'Mean':<12} {'Std Dev':<12} {'Min':<12} {'Max':<12}")
print("-" * 60)
for year in years_to_plot:
    if year in monthly_stats.index:
        stats = monthly_stats.loc[year]
        print(f"{year:<8} {stats['mean']:>10.2f}   {stats['std']:>10.2f}   "
              f"{stats['min']:>10.2f}   {stats['max']:>10.2f}")

# Seasonal analysis
print("\n--- SEASONAL PATTERNS (All Years Combined) ---")
season_mapping = {12: 'Winter', 1: 'Winter', 2: 'Winter', 3: 'Spring',
                  4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer',
                  8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall'}
df_ghg['season'] = df_ghg['month'].map(season_mapping)
seasonal_avg = df_ghg.groupby('season')['sum'].mean()

season_order = ['Winter', 'Spring', 'Summer', 'Fall']
print(f"{'Season':<12} {'Average Emissions (gCO2eq/kWh)':<30}")
print("-" * 45)
for season in season_order:
    if season in seasonal_avg.index:
        print(f"{season:<12} {seasonal_avg[season]:>25.2f}")

# Identify cleanest and dirtiest periods
print("\n--- BEST AND WORST EMISSION PERIODS ---")
for year in years_to_plot:
    year_data = df_ghg_daily[df_ghg_daily['year'] == year]
    if not year_data.empty:
        min_val = year_data['sum'].min()
        max_val = year_data['sum'].max()
        min_day = year_data.loc[year_data['sum'].idxmin(), 'day_of_year']
        max_day = year_data.loc[year_data['sum'].idxmax(), 'day_of_year']

        min_date = pd.Timestamp(year=year, month=1, day=1) + pd.Timedelta(days=int(min_day) - 1)
        max_date = pd.Timestamp(year=year, month=1, day=1) + pd.Timedelta(days=int(max_day) - 1)

        print(f"{year}: Cleanest = {min_val:.2f} gCO2eq/kWh (~{min_date.strftime('%b %d')}) | "
              f"Dirtiest = {max_val:.2f} gCO2eq/kWh (~{max_date.strftime('%b %d')})")

# Year-over-year change in emissions
print("\n--- YEAR-OVER-YEAR CHANGES IN AVERAGE EMISSIONS ---")
for i in range(len(years_to_plot) - 1):
    year1, year2 = years_to_plot[i], years_to_plot[i + 1]
    if year1 in annual_mean.index and year2 in annual_mean.index:
        change_pct = ((annual_mean[year2] - annual_mean[year1]) / annual_mean[year1]) * 100
        change_abs = annual_mean[year2] - annual_mean[year1]
        print(f"{year1} → {year2}: {change_pct:+.2f}% ({change_abs:+.2f} gCO2eq/kWh)")

# Percentile analysis (distribution)
print("\n--- EMISSIONS DISTRIBUTION (All Years) ---")
percentiles = [10, 25, 50, 75, 90]
print(f"{'Percentile':<12} {'Value (gCO2eq/kWh)':<20}")
print("-" * 35)
for p in percentiles:
    val = df_ghg['sum'].quantile(p / 100)
    print(f"{p}th{'':<8} {val:>15.2f}")

# Time spent in different emission ranges
print("\n--- TIME SPENT IN EMISSION RANGES (2016-2024 Combined) ---")
ranges = [
    ('Very Low', 0, 20),
    ('Low', 20, 40),
    ('Medium', 40, 60),
    ('High', 60, 80),
    ('Very High', 80, float('inf'))
]

total_hours = len(df_ghg)
print(f"{'Range':<15} {'Threshold (gCO2eq/kWh)':<25} {'Hours':<12} {'%':<8}")
print("-" * 65)
for label, low, high in ranges:
    count = len(df_ghg[(df_ghg['sum'] >= low) & (df_ghg['sum'] < high)])
    pct = (count / total_hours) * 100
    print(f"{label:<15} {low:>5.0f} - {high:>5.0f}{'':<10} {count:>10,}   {pct:>6.2f}%")

# Volatility by year
print("\n--- EMISSIONS VOLATILITY BY YEAR (Coefficient of Variation) ---")
for year in years_to_plot:
    year_data = df_ghg_daily[df_ghg_daily['year'] == year]
    if not year_data.empty:
        cv = (year_data['sum'].std() / year_data['sum'].mean()) * 100
        print(f"{year}: {cv:.2f}% (lower = more stable)")

# Best performing year
print("\n--- BEST AND WORST PERFORMING YEARS ---")
best_year = annual_mean.idxmin()
worst_year = annual_mean.idxmax()
print(f"Cleanest Year: {best_year} ({annual_mean[best_year]:.2f} gCO2eq/kWh)")
print(f"Dirtiest Year: {worst_year} ({annual_mean[worst_year]:.2f} gCO2eq/kWh)")
print(f"Difference: {annual_mean[worst_year] - annual_mean[best_year]:.2f} gCO2eq/kWh "
      f"({((annual_mean[worst_year] - annual_mean[best_year]) / annual_mean[best_year] * 100):.1f}%)")

print("=" * 80)

SWISS CONSUMPTION MIX GHG EMISSIONS (2016-2024) - STATISTICAL SUMMARY

--- ANNUAL AVERAGE EMISSIONS INTENSITY (gCO2eq/kWh) ---
2018: Mean = 126.86 gCO2eq/kWh | Median = 119.39 gCO2eq/kWh
2019: Mean = 97.25 gCO2eq/kWh | Median = 85.72 gCO2eq/kWh
2020: Mean = 86.43 gCO2eq/kWh | Median = 79.11 gCO2eq/kWh
2021: Mean = 117.12 gCO2eq/kWh | Median = 103.11 gCO2eq/kWh
2022: Mean = 134.51 gCO2eq/kWh | Median = 125.97 gCO2eq/kWh
2023: Mean = 80.37 gCO2eq/kWh | Median = 68.43 gCO2eq/kWh
2024: Mean = 58.01 gCO2eq/kWh | Median = 50.63 gCO2eq/kWh

Change (2016 → 2024): -61.68% (-93.38 gCO2eq/kWh)

--- DAILY AVERAGE EMISSIONS (gCO2eq/kWh) ---
Year     Mean         Std Dev      Min          Max         
------------------------------------------------------------
2018         126.86        57.84        36.92       259.78
2019          97.25        43.83        33.93       252.36
2020          86.43        32.33        37.03       216.35
2021         117.12        51.34        35.04       296.48
2022  

In [6]:
# --- 1. Préparation des Données ---
# On s'assure d'avoir les données propres
df_ghg_source = tot_electricity_impact_CH.loc['2018':'2024'].copy()
df_cons = tot_consumption_CH.loc['2018':'2024'].copy() / 1000
hdd_source = pd.DataFrame(meteo_data['HDD'].copy())
hdd_source.columns = ['hdd']

# Fonction pour assigner les saisons (Hiver = Déc, Jan, Fév, Mar)
def assign_season(month):
    if month in [12, 1, 2, 3]:
        return 'Winter'
    elif month in [6, 7, 8, 9]:
        return 'Summer'
    else:
        return 'Transition'

# Application du mapping saisonnier
df_ghg_source['season'] = df_ghg_source.index.month.map(assign_season)
hdd_source['season'] = hdd_source.index.month.map(assign_season)
df_cons['season'] = df_cons.index.month.map(assign_season)

ghg_winter = df_ghg_source[df_ghg_source['season'] == 'Winter']['sum']
ghg_summer = df_ghg_source[df_ghg_source['season'] == 'Summer']['sum']
cons_winter = df_cons[df_cons['season'] == 'Winter']['sum']
cons_summer = df_cons[df_cons['season'] == 'Summer']['sum']
hdd_winter = hdd_source[hdd_source['season'] == 'Winter']['hdd']
hdd_summer = hdd_source[hdd_source['season'] == 'Summer']['hdd']

ghg_winter.index = ghg_winter.index.strftime('%Y')
ghg_summer.index = ghg_summer.index.strftime('%Y')
hdd_winter.index = hdd_winter.index.strftime('%Y')
hdd_summer.index = hdd_summer.index.strftime('%Y')

# ===== GRAPH 1: BOXPLOT =====
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=['Winter',
                    'Summer'],
    vertical_spacing=0.20,
    specs=[[{"secondary_y": True}], [{"secondary_y": True}]]
)

for i, year in enumerate(years_to_plot):

    fig.add_trace(
        go.Box(
            y=ghg_winter.loc[str(year)],
            name=str(year),
            line=dict(width=1,color=color[str(year)]),
            marker=dict(color=color[str(year)], opacity=0.8),
            boxmean=True,
            showlegend=False
        ), row=1, col=1
    )

    fig.add_trace(
            go.Scatter(
                x=[str(year)],
                y=[hdd_winter.loc[str(year)].sum()],
                name=f'HDD {year}',
                mode='markers',
                marker=dict(
                    symbol='circle',
                    size=7,
                    color='blue',
                    line=dict(width=1, color='darkblue')
                ),
                showlegend=False,
            ),row=1, col=1,
            secondary_y=True
        )

    fig.add_trace(
        go.Box(
            y=ghg_summer.loc[str(year)],
            name=str(year),
            line=dict(width=1,color=color[str(year)]),
            marker=dict(color=color[str(year)], opacity=0.8),
            boxmean=True,
            showlegend=False
        ), row=2, col=1
    )

    fig.add_trace(
            go.Scatter(
                x=[str(year)],
                y=[hdd_summer.loc[str(year)].sum()],
                name=f'HDD',
                mode='markers',
                marker=dict(
                    symbol='circle',
                    size=7,
                    color='blue',
                    line=dict(width=1, color='darkblue')
                ),
                showlegend=True if year == 2024 else False,
            ),row=2, col=1,
            secondary_y=True
        )

    # Update axes
    fig.update_xaxes(
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=1, col=1
    )

    fig.update_xaxes(
        title_text='Year',
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=2, col=1
    )

    fig.update_yaxes(
        title_text='GHG emissions [gCO2eq/kWh]',
        range=[0, 350],
        dtick=70,
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=1, col=1,secondary_y=False
    )

    fig.update_yaxes(
        title_text='GHG emissions [gCO2eq/kWh]',
        range=[0, 350],
        dtick=70,
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=2, col=1, secondary_y=False
    )

    fig.update_yaxes(
        title_text='HDD',
        range=[0, 2500],
        dtick=500,
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=1, col=1, secondary_y=True
    )

    fig.update_yaxes(
        title_text='HDD',
        range=[0, 2500],
        dtick=500,
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=2, col=1, secondary_y=True
    )

fig.update_layout(
    title=dict(text="Swiss comsumption mix yearly GHG Emissions boxplot and HDD by season",x=0.5, font=dict(size=14, color="black", family="Arial")),
    yaxis_title="GHG emissions [gCO2eq/kWh]",
    xaxis_title="Year",
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    yaxis=dict(rangemode='tozero'),
    font=dict(
        size=9,
        color="black",
        family = 'Arial'
    )
)

fig.update_annotations(font=dict(size=12, color='black', family='Arial'))

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\Partie 2\boxplot_season.png",
                width=591, height=500, scale=3)

fig.show()

# ===== GRAPH 1: BOXPLOT =====
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=['Winter',
                    'Summer'],
    vertical_spacing=0.20,
    specs=[[{"secondary_y": True}], [{"secondary_y": True}]]
)

for i, year in enumerate(years_to_plot):

    fig.add_trace(
        go.Box(
            y=cons_winter.loc[str(year)],
            name=str(year),
            line=dict(width=1,color=color[str(year)]),
            marker=dict(color=color[str(year)], opacity=0.8),
            boxmean=True,
            showlegend=False
        ), row=1, col=1
    )

    fig.add_trace(
            go.Scatter(
                x=[str(year)],
                y=[hdd_winter.loc[str(year)].sum()],
                name=f'HDD {year}',
                mode='markers',
                marker=dict(
                    symbol='circle',
                    size=7,
                    color='blue',
                    line=dict(width=1, color='darkblue')
                ),
                showlegend=False,
            ),row=1, col=1,
            secondary_y=True
        )

    fig.add_trace(
        go.Box(
            y=cons_summer.loc[str(year)],
            name=str(year),
            line=dict(width=1,color=color[str(year)]),
            marker=dict(color=color[str(year)], opacity=0.8),
            boxmean=True,
            showlegend=False
        ), row=2, col=1
    )

    fig.add_trace(
            go.Scatter(
                x=[str(year)],
                y=[hdd_summer.loc[str(year)].sum()],
                name=f'HDD',
                mode='markers',
                marker=dict(
                    symbol='circle',
                    size=7,
                    color='blue',
                    line=dict(width=1, color='darkblue')
                ),
                showlegend=True if year == 2024 else False,
            ),row=2, col=1,
            secondary_y=True
        )

    # Update axes
    fig.update_xaxes(
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=1, col=1
    )

    fig.update_xaxes(
        title_text='Year',
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=2, col=1
    )

    fig.update_yaxes(
        title_text='Energy [GWh]',
        range=[3, 15],
        dtick=3,
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=1, col=1, secondary_y=False
    )

    fig.update_yaxes(
        title_text='Energy [GWh]',
        range=[3, 15],
        dtick=3,
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=2, col=1, secondary_y=False
    )

    fig.update_yaxes(
        title_text='HDD',
        range=[0, 2500],
        dtick=625,
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=1, col=1, secondary_y=True
    )

    fig.update_yaxes(
        title_text='HDD',
        range=[0, 2500],
        dtick=625,
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showline=True,
        linewidth=0.8,
        mirror=True,
        linecolor='lightgray',
        zeroline=True,
        zerolinecolor='lightgray',
        zerolinewidth=0.8,
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=2, col=1, secondary_y=True
    )

fig.update_layout(
    title=dict(text="Swiss comsumption mix yearly boxplot and HDD by season",x=0.5, font=dict(size=14, color="black", family="Arial")),
    yaxis_title="Energy [GWh]",
    xaxis_title="Year",
    template='plotly_white',
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.15,
        xanchor="center",
        x=0.5,
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    yaxis=dict(rangemode='tozero'),
    font=dict(
        size=9,
        color="black",
        family = 'Arial'
    )
)

fig.update_annotations(font=dict(size=12, color='black', family='Arial'))

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\Partie 2\boxplotcons_season.png",
                width=591, height=500, scale=3)

fig.show()

In [7]:
# --- 1. Préparation des données ---
# On garde votre base
df_consumption = tot_consumption_CH.loc['2018':'2024'].copy()
df_yearly_gwh = df_consumption.resample('YE').sum()
df_yearly_gwh = df_yearly_gwh*1e-3

sorted_countries = ['CH', 'FR', 'DE', 'AT', 'IT', 'Other']

# Note : On ne calcule plus df_yearly_pct car on veut les valeurs absolues

# Récupération des couleurs (Assurez-vous que le dict 'color' est défini, sinon remplacez par country_colors)
# color = ...

# --- 2. Création de la Figure ---
fig_mix_annual = make_subplots(specs=[[{"secondary_y": True}]])

# --- 3. Boucle pour les Barres (Volumes) ---
for country in sorted_countries:
    # Vérification directe du nom de colonne
    if country in df_yearly_gwh.columns:
        values = df_yearly_gwh[country].values
        pcts = (df_yearly_gwh[country] / df_yearly_gwh['sum']) * 100

        # Texte au survol : Affiche GWh et %
        hover_text = [f'{country}<br>{p:.1f} % ({v:.1f}GWh)' for v, p in zip(values, pcts)]

        fig_mix_annual.add_trace(
            go.Bar(
                # x=years_to_plot, # Si cette variable n'existe pas, utilisez l'index :
                x=df_yearly_gwh.index.year.astype(str),
                y=values,
                name=country,
                marker=dict(
                    color=color.get(country, 'grey'), # Utilisation de .get par sécurité
                    line=dict(color='white', width=0.5)
                ),
                hovertemplate='%{hovertext}<extra></extra>',
                hovertext=hover_text,

                # Affiche la valeur en GWh si la barre est assez grande (> 500 GWh par ex)
                # Vous pouvez ajuster le seuil
                text=[f'{p:.0f}%' if p > 5 else '' for p in pcts],
                textposition='inside',
                textfont=dict(size=9, color='white'),
                showlegend=True
            ),
            secondary_y=False
        )

# --- 5. Mise en Page ---
fig_mix_annual.update_xaxes(
    title_text='Year',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True
)

# Axe Gauche (Volume GWh)
fig_mix_annual.update_yaxes(
    range=[0, 65000],
    dtick=13000,
    title_text='Energy Consumption [GWh]', # Changement du titre
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True, gridcolor='lightgray', gridwidth=0.5,
    secondary_y=False
)

fig_mix_annual.update_layout(
    template='plotly_white',
    barmode='stack', # Empilement des GWh
    title=dict(
        text='Origins of yearly Swiss consumer mix in Switzerland (2018-2024)',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    legend=dict(
        font=dict(size=10, color='black', family='Arial'),
        orientation='h',
        xanchor='center',
        yanchor='top',
        x=0.5,
        y=-0.15,
        bgcolor='rgba(255,255,255,0)'
    ),
    margin=dict(l=60, r=60, t=60, b=80),
    hovermode='x unified',
    font=dict(size=10, color='black', family='Arial'),
)

fig_mix_annual.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\Partie 3\Consumption_barchart_CH.png",
                width=591, height=333, scale=3)

fig_mix_annual.show()

In [8]:
# --- 1. Préparation des données ---
# On garde votre base
df_emission = tot_electricity_impact_CH.loc['2018':'2024'].copy()
df_yearly_ghg = df_emission.resample('YE').mean()
df_yearly_ghg = df_yearly_ghg

sorted_countries = ['CH', 'FR', 'DE', 'AT', 'IT', 'Other']

# Note : On ne calcule plus df_yearly_pct car on veut les valeurs absolues

# Récupération des couleurs (Assurez-vous que le dict 'color' est défini, sinon remplacez par country_colors)
# color = ...

# --- 2. Création de la Figure ---
fig_mix_annual = make_subplots(specs=[[{"secondary_y": True}]])

# --- 3. Boucle pour les Barres (Volumes) ---
for country in sorted_countries:
    # Vérification directe du nom de colonne
    if country in df_yearly_ghg.columns:
        values = df_yearly_ghg[country].values
        pcts = (df_yearly_ghg[country] / df_yearly_ghg['sum']) * 100

        # Texte au survol : Affiche GWh et %
        hover_text = [f'{country}<br>{p:.1f} % ({v:.1f}gCO2eq/kWh)' for v, p in zip(values, pcts)]

        fig_mix_annual.add_trace(
            go.Bar(
                # x=years_to_plot, # Si cette variable n'existe pas, utilisez l'index :
                x=df_yearly_ghg.index.year.astype(str),
                y=values,
                name=country,
                marker=dict(
                    color=color.get(country, 'grey'), # Utilisation de .get par sécurité
                    line=dict(color='white', width=0.5)
                ),
                hovertemplate='%{hovertext}<extra></extra>',
                hovertext=hover_text,
                text=[f'{p:.0f}%' if p > 1 else '' for p in pcts],
                textposition='inside',
                insidetextanchor='middle',
                textfont=dict(size=9, color='white'),
                showlegend=True
            ),
            secondary_y=False
        )

# --- 5. Mise en Page ---
fig_mix_annual.update_xaxes(
    title_text='Year',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True
)

# Axe Gauche (Volume GWh)
fig_mix_annual.update_yaxes(
    title_text='GHG emissions [gCO2eq/kWh]', # Changement du titre
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True, gridcolor='lightgray', gridwidth=0.5,
    secondary_y=False
)

fig_mix_annual.update_layout(
    template='plotly_white',
    barmode='stack', # Empilement des GWh
    title=dict(
        text='Yearly average of GHG emissions in Switzerland by country (2018-2024)',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    legend=dict(
        font=dict(size=10, color='black', family='Arial'),
        orientation='h',
        xanchor='center',
        yanchor='top',
        x=0.5,
        y=-0.15,
        bgcolor='rgba(255,255,255,0)'
    ),
    margin=dict(l=60, r=60, t=60, b=80),
    hovermode='x unified',
    font=dict(size=10, color='black', family='Arial'),
)

fig_mix_annual.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\Partie 3\GHGemissions_barchart_CH.png",
                width=591, height=333, scale=3)

fig_mix_annual.show()

#### c) Meteorological analysis

In [9]:
# Filter Swiss temperature data from 2016 to the end of 2024
temp_data = meteo_data['air_temperature_global'].loc['2016':'2024']

# Split the data by period
temp_2016_2023 = temp_data.loc['2016':'2023']
temp_2024 = temp_data.loc['2024']

# Compute the monthly mean from 2016 to 2023
temp_monthly_avg_2016_2023 = temp_2016_2023.resample('ME').mean()
monthly_avg_global = temp_monthly_avg_2016_2023.mean()

# Compute the annual average for 2024
mean_2024 = temp_2024.mean()

# ===== COMMON PARAMETERS =====
years_to_plot = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22']

# ===== MONTHLY DATA =====
# Add a column for month and year
temp_data = temp_data.reset_index().rename(columns={'index': 'date'})
temp_data['year'] = pd.to_datetime(temp_data['reference_timestamp']).dt.year
temp_data['month'] = pd.to_datetime(temp_data['reference_timestamp']).dt.month

# Group by year and month, then calculate monthly average
temp_monthly = temp_data.groupby(['year', 'month']).mean()

# Create a new index based on month (to overlay years)
temp_monthly = temp_monthly.reset_index()
temp_monthly['date_normalized'] = pd.to_datetime('2000-01-01') + pd.to_timedelta(
    (temp_monthly['month'] - 1) * 30, unit='D')  # Approximation for display

# =======
# GRAPH 1
# =======
fig = go.Figure()

fig.update_xaxes(
    showgrid=True,
    gridcolor="lightgray",
    gridwidth=0.8
)

fig.add_trace(go.Scatter(
    x=temp_2016_2023.index.to_pydatetime(), y=temp_2016_2023.values, name='2016-2023',
    line=dict(width=1)
))

# Horizontal dashed line for 2016–2023
fig.add_trace(go.Scatter(
    x=[temp_2016_2023.index.min(), temp_2016_2023.index.max()],
    y=[monthly_avg_global, monthly_avg_global],
    mode="lines",
    line=dict(color="orange", width=2, dash="dash"),
    name=f"2016–2023 average ({monthly_avg_global:.2f}°C)"
))

fig.add_trace(go.Scatter(
    x=temp_2024.index.to_pydatetime(), y=temp_2024.values, name='2024',
    line=dict(width=1)
))

# Horizontal dashed line for 2024
fig.add_trace(go.Scatter(
    x=[temp_2024.index.min(), temp_2024.index.max()],
    y=[mean_2024, mean_2024],
    mode="lines",
    line=dict(color="green", width=2, dash="dash"),
    name=f"2024 average ({mean_2024:.2f}°C)"
))

fig.update_yaxes(
    title=dict(font=dict(size=12, color="black")),
    tickfont=dict(size=10, color="black")
)

fig.update_layout(
    title=dict(text="Swiss Daily Average air Temperature (2016-2024)",x=0.5, font=dict(size=14, color="black", family="Arial")),
    yaxis_title="Temperature [°C]",
    xaxis_title="Date",
    template='plotly_white',
    legend=dict(
        orientation='h',
        yanchor="top",
        x=0.5,
        y=-0.3,
        xanchor="center",
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    margin=dict(l=60, r=60, t=60, b=100),
    font=dict(
        size=9,
        color="black",
        family = 'Arial'
    )
)

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\2016-2023_temperature.png",width=591,
                height=333, scale=3)

fig.show()
# =======
# GRAPH 2
# =======
fig = go.Figure()

fig.update_xaxes(
    showgrid=True,
    gridcolor="lightgray",
    gridwidth=0.8
)

for i, year in enumerate(years_to_plot):
    df = temp_monthly[temp_monthly['year'] == year]
    df.index = df['date_normalized']
    fig.add_trace(go.Scatter(
        x=df.index.month_name(), y=df['air_temperature_global'], name=str(year),
        line=dict(width=1)
    ))

fig.update_yaxes(
    title=dict(font=dict(size=12, color="black")),
    tickfont=dict(size=10, color="black")
)

fig.update_layout(
    title=dict(text="Swiss Monthly Average Air Temperature (2016-2024)",x=0.5, font=dict(size=14, color="black", family="Arial")),
    yaxis_title="Temperature [°C]",
    xaxis_title="Month",
    template='plotly_white',
    legend=dict(
        orientation='h',
        yanchor="top",
        x=0.5,
        y=-0.3,
        xanchor="center",
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    margin=dict(l=60, r=60, t=60, b=100),
    font=dict(
        size=9,
        color="black",
        family = 'Arial'
    )
)

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\2016-2023_temperature_monthly_avg.png",width=591,
                height=333, scale=3)

fig.show()
# ======================================================================
# STATISTICAL ANALYSIS - SWISS TEMPERATURE AND GHG EMISSIONS (2016–2024)
# ======================================================================

print("=" * 100)
print("SWISS AIR TEMPERATURE & GHG EMISSIONS (2016–2024) - STATISTICAL SUMMARY")
print("=" * 100)

# ========== 1. TEMPERATURE STATISTICS ==========
print("\n--- ANNUAL AVERAGE TEMPERATURE (°C) ---")
annual_temp_mean = temp_data.groupby('year')['air_temperature_global'].mean()
annual_temp_median = temp_data.groupby('year')['air_temperature_global'].median()

for year in years_to_plot:
    if year in annual_temp_mean.index:
        print(f"{year}: Mean = {annual_temp_mean[year]:.2f} °C | "
              f"Median = {annual_temp_median[year]:.2f} °C")

# Temperature change 2016 → 2024
if 2016 in annual_temp_mean.index and 2024 in annual_temp_mean.index:
    temp_2016 = annual_temp_mean[2016]
    temp_2024 = annual_temp_mean[2024]
    change_temp = ((temp_2024 - temp_2016) / temp_2016) * 100
    print(f"\nChange (2016 → 2024): {change_temp:+.2f}% ({temp_2024 - temp_2016:+.2f} °C)")

# Monthly temperature stats
print("\n--- MONTHLY STATISTICS (°C) ---")
monthly_stats_temp = temp_data.groupby(['year', 'month'])['air_temperature_global'].agg(['mean', 'std', 'min', 'max'])
print(f"{'Year':<6} {'Mean':<8} {'Std':<8} {'Min':<8} {'Max':<8}")
print("-" * 50)
for year in years_to_plot:
    if year in monthly_stats_temp.index.get_level_values(0):
        stats = monthly_stats_temp.loc[year].mean()
        print(f"{year:<6} {stats['mean']:>7.2f} {stats['std']:>8.2f} {stats['min']:>8.2f} {stats['max']:>8.2f}")

# ========== 2. CORRELATION BETWEEN TEMPERATURE & GHG ==========
print("\n--- CORRELATION BETWEEN TEMPERATURE AND GHG INTENSITY ---")

# Set it as the index
temp_data = temp_data.set_index('reference_timestamp')

# Resample
temp_monthly_mean = temp_data['air_temperature_global'].resample('ME').mean()
ghg_monthly_mean = df_ghg['sum'].resample('ME').mean()

# Align datasets
combined = pd.concat([temp_monthly_mean, ghg_monthly_mean], axis=1)
combined.columns = ['temperature', 'ghg']
corr = combined.corr().iloc[0, 1]
print(f"Correlation between monthly temperature and GHG intensity: {corr:.3f}")

if corr < 0:
    print("→ Negative correlation: colder months tend to show higher GHG emissions.")
elif corr > 0:
    print("→ Positive correlation: warmer months coincide with higher GHG emissions.")
else:
    print("→ No clear correlation observed.")

# ========== 3. SEASONAL ANALYSIS ==========
print("\n--- SEASONAL COMPARISON (Temperature vs GHG) ---")
season_mapping = {12: 'Winter', 1: 'Winter', 2: 'Winter',
                  3: 'Spring', 4: 'Spring', 5: 'Spring',
                  6: 'Summer', 7: 'Summer', 8: 'Summer',
                  9: 'Fall', 10: 'Fall', 11: 'Fall'}

temp_data['season'] = temp_data['month'].map(season_mapping)
df_ghg['season'] = df_ghg['month'].map(season_mapping)

seasonal_avg_temp = temp_data.groupby('season')['air_temperature_global'].mean()
seasonal_avg_ghg = df_ghg.groupby('season')['sum'].mean()

print(f"{'Season':<10} {'Avg Temp (°C)':<20} {'Avg GHG (gCO2eq/kWh)':<25}")
print("-" * 55)
for season in ['Winter', 'Spring', 'Summer', 'Fall']:
    if season in seasonal_avg_temp.index and season in seasonal_avg_ghg.index:
        print(f"{season:<10} {seasonal_avg_temp[season]:>10.2f} {seasonal_avg_ghg[season]:>20.2f}")

# ========== 4. TEMPERATURE VOLATILITY ==========
print("\n--- TEMPERATURE VOLATILITY BY YEAR (Coefficient of Variation) ---")
cv_temp = (temp_data.groupby('year')['air_temperature_global'].std() /
           temp_data.groupby('year')['air_temperature_global'].mean()) * 100

for year in years_to_plot:
    if year in cv_temp.index:
        print(f"{year}: {cv_temp[year]:.2f}% (lower = more stable)")

SWISS AIR TEMPERATURE & GHG EMISSIONS (2016–2024) - STATISTICAL SUMMARY

--- ANNUAL AVERAGE TEMPERATURE (°C) ---
2016: Mean = 6.81 °C | Median = 6.62 °C
2017: Mean = 6.79 °C | Median = 6.75 °C
2018: Mean = 7.51 °C | Median = 8.21 °C
2019: Mean = 7.18 °C | Median = 7.35 °C
2020: Mean = 7.65 °C | Median = 7.85 °C
2021: Mean = 6.49 °C | Median = 6.55 °C
2022: Mean = 8.26 °C | Median = 8.15 °C
2023: Mean = 7.98 °C | Median = 7.72 °C
2024: Mean = 8.05 °C | Median = 8.07 °C

Change (2016 → 2024): +18.20% (+1.24 °C)

--- MONTHLY STATISTICS (°C) ---
Year   Mean     Std      Min      Max     
--------------------------------------------------
2016      6.80     3.64     0.11    14.15
2017      6.75     3.77    -0.36    13.37
2018      7.44     3.33     0.07    13.16
2019      7.16     3.46     0.68    13.80
2020      7.64     3.52     0.47    14.29
2021      6.47     3.60    -0.00    13.46
2022      8.21     3.57     1.60    15.11
2023      7.94     3.89     0.45    14.79
2024      8.04     3.6

### I.2. Seasonal analysis
#### a) Typical days by season

In [10]:
# ===== SEASONAL PROFILES: WINTER vs SUMMER COMPARISON FOR 2024 =====

# Filter data for 2024 only
df_2024_consumption = tot_consumption_CH.loc['2024'].copy() / 1000 # Convert MWh to GWh
df_2024_emissions = tot_electricity_impact_CH.loc['2024'].copy()
meteo_2024 = meteo.loc['2024'].copy()

# Define seasons based on months
# Winter: December (previous year), January, February, March
# Summer: June, July, August, September
def assign_season(month):
    if month in [12, 1, 2, 3]:
        return 'Winter'
    elif month in [6, 7, 8, 9]:
        return 'Summer'
    else:
        return 'Transition'

# Add season column
df_2024_consumption['season'] = df_2024_consumption.index.month.map(assign_season)
df_2024_emissions['season'] = df_2024_emissions.index.month.map(assign_season)
meteo_2024['season'] = meteo_2024.index.month.map(assign_season)

# Add hour of day for hourly profiles
df_2024_consumption['hour'] = df_2024_consumption.index.hour
df_2024_emissions['hour'] = df_2024_emissions.index.hour
meteo_2024['hour'] = meteo_2024.index.hour

# ===== GRAPH 1: HOURLY PROFILE COMPARISON - CONSUMPTION =====
# Winter consumption profile
winter_consumption = df_2024_consumption[df_2024_consumption['season'] == 'Winter'].groupby('hour')['sum'].mean()
winter_meteo = meteo_2024[meteo_2024['season'] == 'Winter'].groupby('hour')['air_temperature_global'].mean()
# Summer consumption profile
summer_consumption = df_2024_consumption[df_2024_consumption['season'] == 'Summer'].groupby('hour')['sum'].mean()
summer_meteo = meteo_2024[meteo_2024['season'] == 'Summer'].groupby('hour')['air_temperature_global'].mean()

# Winter emissions profile
winter_emissions = df_2024_emissions[df_2024_emissions['season'] == 'Winter'].groupby('hour')['sum'].mean()
# Summer emissions profile
summer_emissions = df_2024_emissions[df_2024_emissions['season'] == 'Summer'].groupby('hour')['sum'].mean()

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Average Hourly Consumption Profile','Average Hourly GHG Emissions Profile'],
    vertical_spacing=0.1,
    horizontal_spacing=0.15,
    specs=[[{"secondary_y": True}, {"secondary_y": True}]]
)

fig.update_annotations(font=dict(size=12, color='black', family='Arial'))

# Définir les données pour chaque subplot
data_configs = [
    {
        'col': 1,
        'df1': winter_consumption,
        'df2': summer_consumption,
        'y_title': 'Consumption (GWh)'
    },
    {
        'col': 2,
        'df1': winter_emissions,
        'df2': summer_emissions,
        'y_title': 'GHG emissions (gCO2eq/kWh)'
    }
]

for config in data_configs:
    col = config['col']
    df1 = config['df1']
    df2 = config['df2']

    # Left axis
    fig.add_trace(
        go.Scatter(
            x=df1.index,
            y=df1.values,
            line=dict(width=1,color='blue'),
            name='Winter (Dec–Mar)',
            showlegend=(col==1)),
        row=1, col=col, secondary_y=False
    )
    fig.add_trace(
        go.Scatter(
            x=df2.index,
            y=df2.values,
            line=dict(width=1,color='red'),
            name='Summer (Jun–Sep)',
            showlegend=(col==1)),
        row=1, col=col, secondary_y=False
    )

    # Right axis - Temperature
    fig.add_trace(
        go.Scatter(
            x=winter_meteo.index,
            y=winter_meteo.values if hasattr(winter_meteo, 'values') else winter_meteo,
            line=dict(width=1, dash='dash',color='blue'),
            name='Temperature (Dec–Mar)',
            showlegend=(col==1)),
        row=1, col=col, secondary_y=True
    )
    fig.add_trace(
        go.Scatter(
            x=summer_meteo.index,
            y=summer_meteo.values if hasattr(summer_meteo, 'values') else summer_meteo,
            line=dict(width=1, dash='dash',color='red'),
            name='Temperature (Jun–Sep)',
            showlegend=(col==1)),
        row=1, col=col, secondary_y=True
    )

    # Update axes
    fig.update_xaxes(
        title_text='Hour',
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=1, col=col
    )

    fig.update_yaxes(
        title_text=config['y_title'],
        title_font=dict(size=12, color='black', family='Arial'),
        tickfont=dict(size=10, color='black', family='Arial'),
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.8,
        row=1, col=col, secondary_y=False
    )

    if col == 2:
        fig.update_yaxes(
            title_text='Temperature (°C)',
            title_font=dict(size=12, color='black', family='Arial'),
            tickfont=dict(size=10, color='black', family='Arial'),
            showgrid=False,
            row=1, col=col, secondary_y=True
        )

fig.update_layout(
    title=dict(
        text='Hourly Average Consumption and GHG Emissions Profile - Winter vs Summer 2024',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    template='plotly_white',
    showlegend=True,
    legend=dict(
        orientation='h',
        yanchor="top",
        x=0.5,
        y=-0.3,
        xanchor="center",
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    margin=dict(l=60, r=60, t=60, b=100),
    font=dict(
        size=9,
        color='black',
        family='Arial'
    ),
    hovermode='closest'
)

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\meteo_ghg_subplots.png",
                width=591, height=333, scale=3)

fig.show()

# ===== GRAPH 3: BOX PLOTS COMPARISON =====
# Prepare data for boxplots
winter_cons = df_2024_consumption[df_2024_consumption['season'] == 'Winter']['sum']
summer_cons = df_2024_consumption[df_2024_consumption['season'] == 'Summer']['sum']
winter_emis = df_2024_emissions[df_2024_emissions['season'] == 'Winter']['sum']
summer_emis = df_2024_emissions[df_2024_emissions['season'] == 'Summer']['sum']

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Distribution of Hourly Consumption',
                    'Distribution of Hourly GHG Emissions'],
    horizontal_spacing=0.1
)

# Consumption boxplots
fig.add_trace(
    go.Box(
        y=winter_cons,
        name='Winter',
        marker=dict(color='#2E86AB', opacity=0.7),
        boxmean=True,
        showlegend=False
    ),
    row=1, col=1
)

fig.add_trace(
    go.Box(
        y=summer_cons,
        name='Summer',
        marker=dict(color='#E63946', opacity=0.7),
        boxmean=True,
        showlegend=False
    ),
    row=1, col=1
)

# Emissions boxplots
fig.add_trace(
    go.Box(
        y=winter_emis,
        name='Winter',
        marker=dict(color='#2E86AB', opacity=0.7),
        boxmean=True,
        showlegend=False
    ),
    row=1, col=2
)

fig.add_trace(
    go.Box(
        y=summer_emis,
        name='Summer',
        marker=dict(color='#E63946', opacity=0.7),
        boxmean=True,
        showlegend=False
    ),
    row=1, col=2
)

# Update axes
fig.update_yaxes(
    title_text='Consumption (MWh)',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
    row=1, col=1
)

fig.update_yaxes(
    title_text='GHG Emissions (gCO2eq/kWh)',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
    row=1, col=2
)

# Update layout
fig.update_layout(
    title=dict(text="Distribution of Hourly Consumption and GHG Emissions - Winter vs Summer 2024",x=0.5, font=dict(size=14, color="black", family="Arial")),
    template='plotly_white',
    margin=dict(l=80, r=20, t=50, b=50),
    hovermode='closest',
    font=dict(size=9, color='black', family='Arial')
)

# Update subplot titles font
fig.update_annotations(font=dict(size=12, color='black', family='Arial'))

fig.add_annotation(
    xref="paper", x=-0.2,
    y=winter_cons.mean(),
    xanchor="right",
    text=f"{winter_cons.mean():.1f}",
    font=dict(size=8, color='#2E86AB'),
    showarrow=False,
    xshift=-5,
    row=1, col=1
)

fig.add_annotation(
    xref="paper", x=0.8,
    y=summer_cons.mean(),
    xanchor="right",
    text=f"{summer_cons.mean():.1f}",
    font=dict(size=8, color='#E63946'),
    showarrow=False,
    xshift=-5,
    row=1, col=1
)

fig.add_annotation(
    xref="paper", x=-0.2,
    y=winter_emis.mean(),
    xanchor="right",
    text=f"{winter_emis.mean():.1f}",
    font=dict(size=8, color='#2E86AB'),
    showarrow=False,
    xshift=-5,
    row=1, col=2
)

fig.add_annotation(
    xref="paper", x=0.8,
    y=summer_emis.mean(),
    xanchor="right",
    text=f"{summer_emis.mean():.1f}",
    font=dict(size=8, color='#E63946'),
    showarrow=False,
    xshift=-5,
    row=1, col=2
)

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\boxplot_winter_summer.png",
                width=591, height=333, scale=3)

fig.show()

# ===== STATISTICAL SUMMARY =====
print("=" * 60)
print("SEASONAL COMPARISON 2024 - STATISTICAL SUMMARY")
print("=" * 60)
print("\n--- CONSUMPTION (MWh) ---")
print(f"Winter - Mean: {winter_cons.mean():.2f} | Median: {winter_cons.median():.2f} | Std: {winter_cons.std():.2f}")
print(f"Summer - Mean: {summer_cons.mean():.2f} | Median: {summer_cons.median():.2f} | Std: {summer_cons.std():.2f}")
print(f"Difference: {((winter_cons.mean() - summer_cons.mean()) / summer_cons.mean() * 100):.1f}% higher in winter")

print("\n--- GHG EMISSIONS (gCO2eq/kWh) ---")
print(f"Winter - Mean: {winter_emis.mean():.2f} | Median: {winter_emis.median():.2f} | Std: {winter_emis.std():.2f}")
print(f"Summer - Mean: {summer_emis.mean():.2f} | Median: {summer_emis.median():.2f} | Std: {summer_emis.std():.2f}")
print(f"Difference: {((winter_emis.mean() - summer_emis.mean()) / summer_emis.mean() * 100):.1f}% higher in winter")
print("=" * 60)

SEASONAL COMPARISON 2024 - STATISTICAL SUMMARY

--- CONSUMPTION (MWh) ---
Winter - Mean: 7.69 | Median: 7.66 | Std: 0.85
Summer - Mean: 6.38 | Median: 6.38 | Std: 0.93
Difference: 20.7% higher in winter

--- GHG EMISSIONS (gCO2eq/kWh) ---
Winter - Mean: 75.44 | Median: 71.24 | Std: 27.86
Summer - Mean: 42.84 | Median: 41.82 | Std: 11.66
Difference: 76.1% higher in winter


## II. Mix analysis
### II.1. Consumption analysis
#### a) Local vs Imports Production

In [11]:
# ===== LOCAL PRODUCTION vs IMPORTS ANALYSIS FOR 2024 =====

# Filter data for 2024
df_2024_consumption = tot_consumption_CH.loc['2024'].copy()
df_2024_impact_by_src = electricity_impact_by_src_CH.loc['2024'].copy()

# Calculate local production (CH) and imports (all other countries)
local_production = df_2024_consumption['CH'].resample('ME').sum() / 1000  # Convert to GWh

# Calculate total imports
import_cols = [col for col in df_2024_consumption.columns if col != 'CH' and col != 'sum']
total_imports = df_2024_consumption[import_cols].sum(axis=1).resample('ME').sum() / 1000

# ===== CALCULATE GHG EMISSIONS CORRECTLY =====
# For local production: use only CH sources from impact data
local_impact_cols = [col for col in df_2024_impact_by_src.columns if col.endswith('_CH')]
# For imports: use all non-CH sources
import_impact_cols = [col for col in df_2024_impact_by_src.columns
                      if not col.endswith('_CH') and col != 'sum']

# Calculate weighted average GHG emissions
# Weighted by energy consumption from each source
local_consumption_hourly = df_2024_consumption[['CH']]  # Only CH column
import_consumption_hourly = df_2024_consumption[import_cols]

# For local production: weight by consumption from each CH source
local_impact_hourly = df_2024_impact_by_src[local_impact_cols]
# Average GHG of local production (weighted by energy from each local source)
local_ghg_monthly = local_impact_hourly.mean(axis=1).resample('ME').mean()

# For imports: weight by consumption from each import source
import_impact_hourly = df_2024_impact_by_src[import_impact_cols]
# Average GHG of imports (weighted by energy from each import source)
import_ghg_monthly = import_impact_hourly.mean(axis=1).resample('ME').mean()

# ===== GRAPH 1: MONTHLY EVOLUTION - PRODUCTION vs IMPORTS =====
fig1 = go.Figure()

x_months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
            'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig1.add_trace(go.Bar(
    x=x_months,
    y=local_production.values,
    name='Local Production',
    marker=dict(color='#2E86AB', opacity=0.8),
    width=0.35
))

fig1.add_trace(go.Bar(
    x=x_months,
    y=total_imports.values,
    name='Total Imports',
    marker=dict(color='#E63946', opacity=0.8),
    width=0.35
))

fig1.update_xaxes(
    title_text='Month',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial')
)

fig1.update_yaxes(
    title_text='Energy (GWh)',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8
)

fig1.update_layout(
    title=dict(
        text='Swiss Energy Mix 2024: Local Production vs Imports (Monthly)',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    template='plotly_white',
    barmode='group',
    legend=dict(
        orientation='h',
        yanchor="top",
        x=0.5,
        y=-0.3,
        xanchor="center",
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    margin=dict(l=60, r=60, t=60, b=100),
    font=dict(size=9, color='black', family='Arial'),
)

fig1.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\production_vs_imports.png",
                 width=591, height=333, scale=3)
fig1.show()


# ===== GRAPH 2: SELF-SUFFICIENCY RATIO =====
fig2 = go.Figure()

total_consumption = local_production + total_imports
self_sufficiency_ratio = (local_production / total_consumption * 100)

# Line with markers
fig2.add_trace(go.Scatter(
    x=list(range(1, 13)),
    y=self_sufficiency_ratio.values,
    mode='lines+markers',
    name='Self-Sufficiency Ratio',
    line=dict(color='#06A77D', width=3),
    marker=dict(size=10, color='#06A77D')
))

# 50% threshold line
fig2.add_trace(go.Scatter(
    x=[1, 12],
    y=[50, 50],
    mode='lines',
    name='50% Threshold',
    line=dict(color='red', width=2, dash='dash'),
    opacity=0.7
))

# Fill areas above 50%
fig2.add_trace(go.Scatter(
    x=list(range(1, 13)),
    y=self_sufficiency_ratio.values,
    fill='tonexty',
    fillcolor='rgba(144, 238, 144, 0.3)',
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))

# Fill areas below 50%
y_below = [min(val, 50) for val in self_sufficiency_ratio.values]
fig2.add_trace(go.Scatter(
    x=list(range(1, 13)),
    y=y_below,
    fill='tonexty',
    fillcolor='rgba(0, 200, 0, 0.3)',
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))

# Add text annotations
for i, val in enumerate(self_sufficiency_ratio.values):
    fig2.add_annotation(
        x=i+1,
        y=val + 2,
        text=f'{val:.1f}%',
        showarrow=False,
        font=dict(size=9, color='black', family='Arial')
    )

fig2.update_xaxes(
    title_text='Month',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    tickmode='array',
    tickvals=list(range(1, 13)),
    ticktext=x_months,
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8
)

fig2.update_yaxes(
    title_text='Self-Sufficiency Ratio (%)',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    range=[0, 100],
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8
)

fig2.update_layout(
    title=dict(
        text='Swiss Energy Self-Sufficiency Ratio 2024',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    template='plotly_white',
    legend=dict(
        orientation='h',
        yanchor="top",
        x=0.5,
        y=-0.3,
        xanchor="center",
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    margin=dict(l=60, r=60, t=60, b=100),
    font=dict(size=9, color='black', family='Arial'),
)

fig2.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\self_sufficiency_ratio.png",
                 width=591, height=333, scale=3)
fig2.show()


# ===== GRAPH 3: GHG EMISSIONS COMPARISON =====
fig3 = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Average GHG Emissions Intensity',
                    'Cumulative GHG Emissions 2024'],
    horizontal_spacing=0.12
)

# SUBPLOT 1: Monthly GHG emissions intensity comparison
fig3.add_trace(
    go.Bar(
        x=x_months,
        y=local_ghg_monthly.values,
        name='Local Production',
        marker=dict(color='#2E86AB', opacity=0.8),
        width=0.35
    ),
    row=1, col=1
)

fig3.add_trace(
    go.Bar(
        x=x_months,
        y=import_ghg_monthly.values,
        name='Imports',
        marker=dict(color='#E63946', opacity=0.8),
        width=0.35
    ),
    row=1, col=1
)

# SUBPLOT 2: Cumulative GHG emissions
local_production_mwh = df_2024_consumption['CH'].resample('ME').sum()
imports_mwh = df_2024_consumption[import_cols].sum(axis=1).resample('ME').sum()

local_total_ghg = (local_ghg_monthly * local_production_mwh / 1000).cumsum()
import_total_ghg = (import_ghg_monthly * imports_mwh / 1000).cumsum()

fig3.add_trace(
    go.Scatter(
        x=x_months,
        y=local_total_ghg.values,
        mode='lines+markers',
        name='Local Production',
        line=dict(color='#2E86AB', width=3),
        marker=dict(size=8, symbol='circle'),
        showlegend=False
    ),
    row=1, col=2
)

fig3.add_trace(
    go.Scatter(
        x=x_months,
        y=import_total_ghg.values,
        mode='lines+markers',
        name='Imports',
        line=dict(color='#E63946', width=3),
        marker=dict(size=8, symbol='square'),
        showlegend=False
    ),
    row=1, col=2
)

# Update axes
fig3.update_xaxes(
    title_text='Month',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    row=1, col=1
)

fig3.update_xaxes(
    title_text='Month',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    row=1, col=2
)

fig3.update_yaxes(
    title_text='GHG emissions (gCO2eq/kWh)',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
    row=1, col=1
)

fig3.update_yaxes(
    title_text='GHG Emissions (tons CO2eq)',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
    row=1, col=2
)

# Update layout
fig3.update_layout(
    title=dict(
        text='GHG Emissions Analysis : Local vs Imports',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    template='plotly_white',
    barmode='group',
    legend=dict(
        orientation='h',
        yanchor="top",
        x=0.5,
        y=-0.3,
        xanchor="center",
        font=dict(size=10, color="black"),
        bgcolor="rgba(255,255,255,0)"
    ),
    margin=dict(l=60, r=60, t=60, b=100),
    font=dict(size=9, color='black', family='Arial'),
)

fig3.update_annotations(font=dict(size=12, color='black', family='Arial'))

fig3.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\ghg_emissions_comparison.png",
                 width=591, height=333, scale=3)
fig3.show()

# ===== STATISTICAL SUMMARY =====
print("=" * 70)
print("PRODUCTION vs IMPORTS ANALYSIS 2024 - SUMMARY")
print("=" * 70)

print("\n--- ENERGY CONSUMPTION (GWh) ---")
print(f"Total Local Production: {local_production.sum():.2f} GWh")
print(f"Total Imports: {total_imports.sum():.2f} GWh")
print(f"Total Consumption: {(local_production.sum() + total_imports.sum()):.2f} GWh")
print(f"Average Self-Sufficiency Ratio: {self_sufficiency_ratio.mean():.1f}%")
print(f"Highest Self-Sufficiency: {self_sufficiency_ratio.max():.1f}% (Month {self_sufficiency_ratio.idxmax().month})")
print(f"Lowest Self-Sufficiency: {self_sufficiency_ratio.min():.1f}% (Month {self_sufficiency_ratio.idxmin().month})")

print("\n--- GHG EMISSIONS INTENSITY (gCO2eq/kWh) ---")
print(f"Average Local Production Emissions: {local_ghg_monthly.mean():.2f} gCO2eq/kWh")
print(f"Average Import Emissions: {import_ghg_monthly.mean():.2f} gCO2eq/kWh")
print(f"Difference: {import_ghg_monthly.mean() - local_ghg_monthly.mean():.2f} gCO2eq/kWh")
if local_ghg_monthly.mean() > 0:
    print(f"Imports are {((import_ghg_monthly.mean() / local_ghg_monthly.mean() - 1) * 100):.1f}% more carbon-intensive")

print("\n--- TOTAL GHG EMISSIONS (tons CO2eq) ---")
print(f"Total from Local Production: {local_total_ghg.iloc[-1]:.2f} tons CO2eq")
print(f"Total from Imports: {import_total_ghg.iloc[-1]:.2f} tons CO2eq")
print(f"Grand Total: {(local_total_ghg.iloc[-1] + import_total_ghg.iloc[-1]):.2f} tons CO2eq")
print(f"Import Share of Total Emissions: {(import_total_ghg.iloc[-1] / (local_total_ghg.iloc[-1] + import_total_ghg.iloc[-1]) * 100):.1f}%")
print("=" * 70)

PRODUCTION vs IMPORTS ANALYSIS 2024 - SUMMARY

--- ENERGY CONSUMPTION (GWh) ---
Total Local Production: 45578.47 GWh
Total Imports: 16081.37 GWh
Total Consumption: 61659.84 GWh
Average Self-Sufficiency Ratio: 74.6%
Highest Self-Sufficiency: 90.4% (Month 7)
Lowest Self-Sufficiency: 61.8% (Month 12)

--- GHG EMISSIONS INTENSITY (gCO2eq/kWh) ---
Average Local Production Emissions: 0.66 gCO2eq/kWh
Average Import Emissions: 0.43 gCO2eq/kWh
Difference: -0.23 gCO2eq/kWh
Imports are -34.5% more carbon-intensive

--- TOTAL GHG EMISSIONS (tons CO2eq) ---
Total from Local Production: 30384.74 tons CO2eq
Total from Imports: 8415.99 tons CO2eq
Grand Total: 38800.73 tons CO2eq
Import Share of Total Emissions: 21.7%


#### b) Technologies repartion in Swiss local production

In [12]:
# ===============
# Data processing
# ===============

# Load data
df_impact = electricity_impact_by_src_CH.loc['2024'].copy()
df_prod = raw_consumption_by_src_CH.loc['2024'].copy()

# Filter to keep only columns ending with _CH for production and impacts
df_impact = df_impact[[col for col in df_impact.columns if col.endswith('_CH')]]
df_prod = df_prod[[col for col in df_prod.columns if col.endswith('_CH')]]

# Resample by Month
df_prod_m = df_prod.resample('ME').sum() / 1000# Convert to GWh
df_impact_m = df_impact.resample('ME').mean()

# Calculate monthly total local production and gCO2eq/kWh emissions
monthly_total_prod = df_prod_m.sum(axis=1)
monthly_total_impact = df_impact_m.sum(axis=1)

# Calculate percentage contribution of each technology
df_monthly_pct = df_prod_m.div(monthly_total_prod, axis=0) * 100

# Calculate average percentage for each technology across all months
avg_pct = df_monthly_pct.mean()

# **FIX 1: Filter out technologies with less than 1% contribution on average**
significant_techs = avg_pct[avg_pct > 1.0].index.tolist()
sorted_techs = avg_pct[significant_techs].sort_values(ascending=False).index.tolist()

# Update dataframes to keep only significant technologies
df_monthly_prod_filtered = df_prod_m[significant_techs]
df_monthly_pct_filtered = df_monthly_pct[significant_techs]

# Sort technologies by average percentage (ascending order for stacking)
sorted_techs = avg_pct.sort_values(ascending=False).index.tolist()

# Define color palette for technologies by category
tech_colors = {}

color_map = {
    'Nuclear': plt.cm.Blues,
    'Wind': plt.cm.Greens,
    'Solar': plt.cm.Greens,
    'Hydro': plt.cm.Greens,
    'Biomass': plt.cm.Greens,
    'Geothermal': plt.cm.Greens,
    'Waste': plt.cm.Greens,
    'Fossil': plt.cm.Oranges,
    'Storage': plt.cm.Greys,
}

def get_color_for_tech(tech, index, total):
    for key, cmap in color_map.items():
        if key in tech:
            return cmap(np.linspace(0.4, 0.9, total)[index % total])
    return plt.cm.Greys(0.5)


def rgba_to_rgb_string(rgba):
    """Convert matplotlib RGBA tuple to Plotly rgb string"""
    r, g, b, a = rgba
    return f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})'


for i, tech in enumerate(sorted_techs):
    tech_colors[tech] = get_color_for_tech(tech, i, len(sorted_techs))

tech_colors_plotly = {tech: rgba_to_rgb_string(color) for tech, color in tech_colors.items()}

# =================================================================================================
# GRAPH 1: Stacked bar chart of technologies local production with their percentage of contribution
# =================================================================================================

# Create the figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

year_labels = ['2018','2019','2020','2021','2022','2023','2024']

# Plot each technology as a stacked bar (from lowest to highest percentage)
for tech in sorted_techs:
    values = df_prod_m[tech].values
    percentages = df_monthly_pct[tech].values

    # Calculer le pourcentage moyen pour cette techno
    avg_pct = percentages.mean()

    # Create hover text with percentage
    hover_text = [f'{tech}<br>Production: {val:.2f} GWh<br>Share: {pct:.1f}%'
                  for val, pct in zip(values, percentages)]

    fig.add_trace(
        go.Bar(
            x=year_labels,
            y=values,
            name=tech,
            marker=dict(
                color=tech_colors_plotly.get(tech, 'rgb(128, 128, 128)'),
                line=dict(color='white', width=0.5)
            ),
            hovertemplate='%{hovertext}<extra></extra>',
            hovertext=hover_text,
            text=[f'{pct:.1f}%' if pct > 3 else '' for pct in percentages],
            textposition='inside',
            textfont=dict(size=8, color='black'),
            showlegend=bool(avg_pct >= 1.0)  # Afficher uniquement si moyenne >= 1%
        ),
        secondary_y=False
    )

# Plot carbon intensity as a dashed line on secondary axis
fig.add_trace(
    go.Scatter(
        x=year_labels,
        y=monthly_total_impact.values,
        name='Carbon Intensity',
        mode='lines',
        line=dict(color='red', width=1, dash='dash'),
        showlegend=False,
        hovertemplate='Carbon Intensity: %{y:.2f} gCO2eq/kWh<extra></extra>'
    ),
    secondary_y=True
)

# Update layout
fig.update_xaxes(
    title_text='Month',
    title_font=dict(size=12, color='black'),
    tickfont=dict(size=10, color='black'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8
)

fig.update_yaxes(
    title_text='Local Production (GWh)',
    title_font=dict(size=12, color='black'),
    tickfont=dict(size=10, color='black'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
    secondary_y=False
)

fig.update_yaxes(
    title_text='GHG Emissions (gCO2eq/kWh)',
    title_font=dict(size=12, color='red'),
    tickfont=dict(size=10, color='red'),
    showgrid=False,
    secondary_y=True
)

fig.update_layout(
    title=dict(
        text='Swiss Local Production by Technology Mix - 2024<br>(Excluding Imports)',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    barmode='stack',
    template='plotly_white',
    legend=dict(
        title=dict(text='Technologies (>1% avg)', font=dict(size=10)),
        font=dict(size=8),
        x=1.1,
        y=1,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255,255,255,0.8)',
    ),
    margin=dict(l=60, r=60, t=60, b=60),
    hovermode='closest',
    font=dict(
        size=9,
        color='black',
        family='Arial'
    )
)

fig.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\techno_mix_shares_production.png",
                 width=591, height=400, scale=3)

fig.show()

# ====================================
# GRAPH 2 : TECHNOLOGY MIX PERCENTAGE
# ====================================

# Create stacked bar chart (100% normalized)
fig2 = go.Figure()

year_labels = ['2018','2019','2020','2021','2022','2023','2024']

for tech in sorted_techs:
    percentages = df_monthly_pct[tech].values

    # Calculer le pourcentage moyen pour cette techno
    avg_pct = percentages.mean()

    # Create hover text
    hover_text = [f'{tech}<br>Share: {pct:.1f}%' for pct in percentages]

    fig2.add_trace(
        go.Bar(
            x=year_labels,
            y=percentages,
            name=tech,
            marker=dict(
                color=tech_colors_plotly[tech],
                line=dict(color='white', width=0.5)
            ),
            hovertemplate='%{hovertext}<extra></extra>',
            hovertext=hover_text,
            text=[f'{pct:.0f}%' if pct > 5 else '' for pct in percentages],
            textposition='inside',
            textfont=dict(size=8, color='white'),
            showlegend=bool(avg_pct >= 1.0)
        )
    )

# Update layout
fig2.update_xaxes(
    title_text='Month',
    title_font=dict(size=12, color='black'),
    tickfont=dict(size=10, color='black'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8
)

fig2.update_yaxes(
    title_text='Share of Local Production (%)',
    title_font=dict(size=12, color='black'),
    tickfont=dict(size=10, color='black'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
    range=[0, 100]
)

fig2.update_layout(
    title=dict(
        text='Monthly Technology Mix in Swiss Local Production - 2024<br>(Each Month = 100%)',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    barmode='stack',
    template='plotly_white',
   legend=dict(
        title=dict(text='Technologies (>1% avg)', font=dict(size=10)),
        font=dict(size=8),
        x=1.1,
        y=1,
        xanchor='left',
        yanchor='top',
        bgcolor='rgba(255,255,255,0.8)',
    ),
    margin=dict(l=60, r=60, t=60, b=60),
    hovermode='closest',
    font=dict(
        size=9,
        color='black',
        family='Arial'
    )
)

fig2.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\techno_mix_shares.png",
                 width=591, height=333, scale=3)

fig2.show()

#=======================================
# SUMMARY STATISTICS
#=======================================

print("=" * 80)
print("SWISS LOCAL PRODUCTION 2024 - TECHNOLOGY MIX SUMMARY")
print("=" * 80)

# --- 1. ANNUAL PRODUCTION ---
print("\n--- ANNUAL PRODUCTION OVERVIEW ---")
annual_total = monthly_total_prod.sum()
monthly_avg = monthly_total_prod.mean()
max_month = monthly_total_prod.idxmax().month
min_month = monthly_total_prod.idxmin().month
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

print(f"Total Annual Production: {annual_total:,.2f} GWh")
print(f"Average Monthly Production: {monthly_avg:,.2f} GWh")
print(f"Highest Production Month: {month_names[max_month-1]} ({monthly_total_prod.max():,.2f} GWh)")
print(f"Lowest Production Month: {month_names[min_month-1]} ({monthly_total_prod.min():,.2f} GWh)")
print(f"Production Range: {monthly_total_prod.max() - monthly_total_prod.min():,.2f} GWh " +
      f"({(monthly_total_prod.max() - monthly_total_prod.min()) / monthly_total_prod.mean() * 100:.1f}% of average)")

# --- 2. TECHNOLOGY CONTRIBUTION ---
print("\n--- TECHNOLOGY CONTRIBUTION (Annual Average) ---")
annual_prod_by_tech = df_prod_m.sum()
sorted_by_prod = annual_prod_by_tech.sort_values(ascending=False)

print("Top 5 Technologies:")
for i, (tech, prod) in enumerate(sorted_by_prod.head(5).items(), 1):
    pct = (prod / annual_total) * 100
    print(f"  {i}. {tech}: {prod:,.2f} GWh ({pct:.1f}%)")

# Renewable vs Non-renewable classification
renewable_techs = [col for col in df_prod_m.columns if any(x in col for x in
                  ['Hydro', 'Solar', 'Wind', 'Biomass', 'Geothermal', 'Waste'])]
non_renewable_techs = [col for col in df_prod_m.columns if col not in renewable_techs]

renewable_prod = df_prod_m[renewable_techs].sum().sum()
non_renewable_prod = df_prod_m[non_renewable_techs].sum().sum()

print(f"\nRenewable Energy: {renewable_prod:,.2f} GWh ({renewable_prod/annual_total*100:.1f}%)")
print(f"Non-Renewable Energy: {non_renewable_prod:,.2f} GWh ({non_renewable_prod/annual_total*100:.1f}%)")

# Hydraulic breakdown
hydro_techs = [col for col in df_prod_m.columns if 'Hydro' in col]
if hydro_techs:
    hydro_total = df_prod_m[hydro_techs].sum().sum()
    print(f"\nTotal Hydraulic Production: {hydro_total:,.2f} GWh ({hydro_total/annual_total*100:.1f}%)")
    for tech in hydro_techs:
        if df_prod_m[tech].sum() > 0:
            print(f"  - {tech}: {df_prod_m[tech].sum():,.2f} GWh ({df_prod_m[tech].sum()/hydro_total*100:.1f}% of hydro)")

# Nuclear (if present)
nuclear_techs = [col for col in df_prod_m.columns if 'Nuclear' in col]
if nuclear_techs and df_prod_m[nuclear_techs].sum().sum() > 0:
    nuclear_prod = df_prod_m[nuclear_techs].sum().sum()
    print(f"\nNuclear Production: {nuclear_prod:,.2f} GWh ({nuclear_prod/annual_total*100:.1f}%)")

# --- 3. VARIABILITY & STABILITY ---
print("\n--- PRODUCTION VARIABILITY BY TECHNOLOGY ---")
print(f"{'Technology':<40} {'Avg (GWh)':<12} {'Std Dev':<12} {'CV (%)':<10}")
print("-" * 80)

cv_by_tech = {}
for tech in sorted_techs:
    mean_val = df_prod_m[tech].mean()
    std_val = df_prod_m[tech].std()
    cv = (std_val / mean_val * 100) if mean_val > 0 else float('inf')
    cv_by_tech[tech] = cv
    print(f"{tech:<40} {mean_val:>10.2f}   {std_val:>10.2f}   {cv:>8.2f}")

most_stable = min(cv_by_tech.items(), key=lambda x: x[1])
most_variable = max(cv_by_tech.items(), key=lambda x: x[1] if x[1] != float('inf') else 0)

print(f"\nMost Stable: {most_stable[0]} (CV: {most_stable[1]:.2f}%)")
print(f"Most Variable: {most_variable[0]} (CV: {most_variable[1]:.2f}%)")

# --- 4. CARBON INTENSITY ---
print("\n--- CARBON INTENSITY ANALYSIS ---")
annual_avg_impact = monthly_total_impact.mean()
annual_median_impact = monthly_total_impact.median()
cleanest_month_idx = monthly_total_impact.idxmin().month
dirtiest_month_idx = monthly_total_impact.idxmax().month

print(f"Annual Average: {annual_avg_impact:.2f} gCO2eq/kWh")
print(f"Annual Median: {annual_median_impact:.2f} gCO2eq/kWh")
print(f"Cleanest Month: {month_names[cleanest_month_idx-1]} ({monthly_total_impact.min():.2f} gCO2eq/kWh)")
print(f"Dirtiest Month: {month_names[dirtiest_month_idx-1]} ({monthly_total_impact.max():.2f} gCO2eq/kWh)")
print(f"Range: {monthly_total_impact.max() - monthly_total_impact.min():.2f} gCO2eq/kWh")

# --- 5. SEASONAL PATTERNS ---
print("\n--- SEASONAL PATTERNS ---")
# Define seasons
winter_months = [11, 0, 1, 2]  # Dec, Jan, Feb, Mar (indices 0-11)
summer_months = [5, 6, 7, 8]   # Jun, Jul, Aug, Sep

winter_prod = monthly_total_prod.iloc[winter_months].sum()
summer_prod = monthly_total_prod.iloc[summer_months].sum()
winter_impact = monthly_total_impact.iloc[winter_months].mean()
summer_impact = monthly_total_impact.iloc[summer_months].mean()

print(f"Winter Production (Dec-Mar): {winter_prod:,.2f} GWh")
print(f"Summer Production (Jun-Sep): {summer_prod:,.2f} GWh")
print(f"Seasonal Difference: {((winter_prod - summer_prod) / summer_prod * 100):+.1f}%")

print(f"\nWinter Avg Carbon Intensity: {winter_impact:.2f} gCO2eq/kWh")
print(f"Summer Avg Carbon Intensity: {summer_impact:.2f} gCO2eq/kWh")
print(f"Seasonal Difference: {((winter_impact - summer_impact) / summer_impact * 100):+.1f}%")

print("=" * 80)

SWISS LOCAL PRODUCTION 2024 - TECHNOLOGY MIX SUMMARY

--- ANNUAL PRODUCTION OVERVIEW ---
Total Annual Production: 45,578.47 GWh
Average Monthly Production: 3,798.21 GWh
Highest Production Month: Jul (4,260.95 GWh)
Lowest Production Month: Feb (3,378.13 GWh)
Production Range: 882.83 GWh (23.2% of average)

--- TECHNOLOGY CONTRIBUTION (Annual Average) ---
Top 5 Technologies:
  1. Nuclear_CH: 14,231.54 GWh (31.2%)
  2. Residual_Hydro_Run-of-river_and_poundage_CH: 9,155.08 GWh (20.1%)
  3. Hydro_Water_Reservoir_CH: 8,185.73 GWh (18.0%)
  4. Hydro_Pumped_Storage_CH: 5,518.04 GWh (12.1%)
  5. Solar_CH: 3,791.48 GWh (8.3%)

Renewable Energy: 31,346.93 GWh (68.8%)
Non-Renewable Energy: 14,231.54 GWh (31.2%)

Total Hydraulic Production: 27,200.64 GWh (59.7%)
  - Residual_Hydro_Water_Reservoir_CH: 2,970.22 GWh (10.9% of hydro)
  - Residual_Hydro_Run-of-river_and_poundage_CH: 9,155.08 GWh (33.7% of hydro)
  - Hydro_Pumped_Storage_CH: 5,518.04 GWh (20.3% of hydro)
  - Hydro_Run-of-river_and_pounda

#### c) Technologies production and GHG emissions for imports (DE & FR)

In [13]:
# ===============
# Data processing
# ===============

# Load data for France and Germany
# Load data
df_impact = electricity_impact_by_src_CH.loc['2018':'2024'].copy()
df_prod = raw_consumption_by_src_CH.loc['2018':'2024'].copy()

# Filter to keep only columns ending with _FR or _DE
df_impact_FR = df_impact[[col for col in df_impact.columns if col.endswith('_FR')]]
df_prod_FR = df_prod[[col for col in df_prod.columns if col.endswith('_FR')]]

df_impact_DE = df_impact[[col for col in df_impact.columns if col.endswith('_DE')]]
df_prod_DE = df_prod[[col for col in df_prod.columns if col.endswith('_DE')]]

# Resample by Year
df_prod_m_FR = df_prod_FR.resample('YE').sum() / 1000  # Convert to GWh
df_impact_m_FR = df_impact_FR.resample('YE').mean()

df_prod_m_DE = df_prod_DE.resample('YE').sum() / 1000  # Convert to GWh
df_impact_m_DE = df_impact_DE.resample('YE').mean()

# Calculate yearly total local production and gCO2eq/kWh emissions
yearly_total_prod_FR = df_prod_m_FR.sum(axis=1)
yearly_total_impact_FR = df_impact_m_FR.sum(axis=1)

yearly_total_prod_DE = df_prod_m_DE.sum(axis=1)
yearly_total_impact_DE = df_impact_m_DE.sum(axis=1)

# Calculate percentage contribution of each technology
df_yearly_pct_FR = df_prod_m_FR.div(yearly_total_prod_FR, axis=0) * 100
df_yearly_pct_DE = df_prod_m_DE.div(yearly_total_prod_DE, axis=0) * 100

# Calculate average percentage for each technology across all months
avg_pct_FR = df_yearly_pct_FR.mean()
avg_pct_DE = df_yearly_pct_DE.mean()

# Filter out technologies with less than 1% contribution on average
significant_techs_FR = avg_pct_FR[avg_pct_FR > 1.0].index.tolist()
significant_techs_DE = avg_pct_DE[avg_pct_DE > 1.0].index.tolist()

# Combine significant technologies from both countries
all_significant_techs = list(set(significant_techs_FR + significant_techs_DE))

# Sort by combined average percentage
combined_avg = (avg_pct_FR.reindex(all_significant_techs, fill_value=0) +
                avg_pct_DE.reindex(all_significant_techs, fill_value=0)) / 2
sorted_techs = combined_avg.sort_values(ascending=False).index.tolist()
existing_techs = sorted(set(df_prod_m_FR.columns) | set(df_prod_m_DE.columns))
sorted_techs = [tech for tech in sorted_techs if tech in existing_techs]

# Define color palette for technologies by category
tech_colors = {}

color_map = {
    'Nuclear': plt.cm.Blues,
    'Wind': plt.cm.Greens,
    'Solar': plt.cm.Greens,
    'Hydro': plt.cm.Greens,
    'Biomass': plt.cm.Greens,
    'Geothermal': plt.cm.Greens,
    'Waste': plt.cm.Greens,
    'Fossil': plt.cm.Oranges,
    'Storage': plt.cm.Greys,
}

def get_color_for_tech(tech, index, total):
    for key, cmap in color_map.items():
        if key in tech:
            return cmap(np.linspace(0.4, 0.9, total)[index % total])
    return plt.cm.Greys(0.5)

def rgba_to_rgb_string(rgba):
    """Convert matplotlib RGBA tuple to Plotly rgb string"""
    r, g, b, a = rgba
    return f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})'


for i, tech in enumerate(sorted_techs):
    tech_colors[tech] = get_color_for_tech(tech, i, len(sorted_techs))

tech_colors_plotly = {tech: rgba_to_rgb_string(color) for tech, color in tech_colors.items()}


# =================================================================================================
# GRAPH 1: yearly Production FR vs DE with GHG on secondary axis
# =================================================================================================

fig1 = make_subplots(specs=[[{"secondary_y": True}]])

year_labels = ['2018','2019','2020','2021','2022','2023','2024']

# Calculate total production by country
total_prod_FR = df_prod_m_FR.sum(axis=1).values
total_prod_DE = df_prod_m_DE.sum(axis=1).values

# Production bars
fig1.add_trace(
    go.Bar(
        x=year_labels,
        y=total_prod_FR,
        name='France',
        marker=dict(color='#2E86AB', opacity=0.8),
        width=0.35,
        hovertemplate='France<br>Exportation: %{y:.2f} GWh<extra></extra>'
    ),
    secondary_y=False
)

fig1.add_trace(
    go.Bar(
        x=year_labels,
        y=total_prod_DE,
        name='Germany',
        marker=dict(color='#E63946', opacity=0.8),
        width=0.35,
        hovertemplate='Germany<br>Exportation: %{y:.2f} GWh<extra></extra>'
    ),
    secondary_y=False
)

# GHG dashed lines on secondary axis
fig1.add_trace(
    go.Scatter(
        x=year_labels,
        y=yearly_total_impact_FR.values,
        name='Carbon Intensity (FR)',
        mode='lines',
        line=dict(color='#2E86AB', width=1, dash='dash'),
        hovertemplate='FR Carbon Intensity: %{y:.2f} gCO2eq/kWh<extra></extra>'
    ),
    secondary_y=True
)

fig1.add_trace(
    go.Scatter(
        x=year_labels,
        y=yearly_total_impact_DE.values,
        name='Carbon Intensity (DE)',
        mode='lines',
        line=dict(color='#E63946', width=1, dash='dash'),
        hovertemplate='DE Carbon Intensity: %{y:.2f} gCO2eq/kWh<extra></extra>'
    ),
    secondary_y=True
)

# Update axes
fig1.update_xaxes(
    title_text='Month',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8
)

fig1.update_yaxes(
    title_text='Exportation to CH (GWh)',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
    secondary_y=False
)

fig1.update_yaxes(
    title_text='Carbon Intensity (gCO2eq/kWh)',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=False,
    secondary_y=True
)

# Update layout
fig1.update_layout(
    title=dict(
        text='France vs Germany - yearly Exportation and Carbon Intensity',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    barmode='group',
    template='plotly_white',
    legend=dict(
        font=dict(size=9, color='black', family='Arial'),
        orientation='h',
        xanchor='center',
        yanchor='top',
        x=0.5,
        y=-0.20,
        bgcolor='rgba(255,255,255,0)'
    ),
    margin=dict(l=60, r=60, t=60, b=100),
    hovermode='closest',
    font=dict(
        size=9,
        color='black',
        family='Arial'
    ))

fig1.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\FR_DE_production_ghg.png",
                 width=591, height=333, scale=3)

fig1.show()

# ====================================
# GRAPH 2: French Exportation to CH Shares Mix
# ====================================

fig_FR = make_subplots(specs=[[{"secondary_y": True}]])

for tech in sorted_techs:
    if tech in df_yearly_pct_FR.columns:
        percentages = df_yearly_pct_FR[tech].values
        avg_pct = percentages.mean()

        hover_text = [f'{tech}<br>Share: {pct:.1f}%' for pct in percentages]

        fig_FR.add_trace(
            go.Bar(
                x=year_labels,
                y=percentages,
                name=tech,
                marker=dict(
                    color=tech_colors_plotly.get(tech, 'rgb(128, 128, 128)'),
                    line=dict(color='white', width=0.5)
                ),
                hovertemplate='%{hovertext}<extra></extra>',
                hovertext=hover_text,
                text=[f'{pct:.0f}%' if pct > 5 else '' for pct in percentages],
                textposition='inside',
                textfont=dict(size=8, color='white'),
                showlegend=bool(avg_pct >= 1.0)
            )
        )

# fig_FR.add_trace(
#     go.Scatter(
#         x=year_labels,
#         y=yearly_total_impact_FR.values,
#         name='Carbon intensity',
#         mode='lines',
#         line=dict(color='black', width=1, dash='dash'),
#         hovertemplate='FR carbon intensity: %{y:.2f} gCO2eq/kWh<extra></extra>'
#     ),
#     secondary_y=True
# )

fig_FR.update_xaxes(
    title_text='Month',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8
)

fig_FR.update_yaxes(
    title_text='French exportation mix for CH (%)',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
    range=[0, 100]
)

# fig_FR.update_yaxes(
#     title_text='Carbon intensity (gCO2eq/kWh)',
#     title_font=dict(size=12, color='black', family='Arial'),
#     tickfont=dict(size=10, color='black', family='Arial'),
#     showgrid=False,
#     secondary_y=True
# )

fig_FR.update_layout(
    template='plotly_white',
    barmode='stack',
    title=dict(
        text='France - Yearly technology mix for exportation',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    legend=dict(
        font=dict(size=9, color='black', family='Arial'),
        orientation='h',
        xanchor='center',
        yanchor='top',
        x=0.5,
        y=-0.20,
        bgcolor='rgba(255,255,255,0)'
    ),
    margin=dict(l=60, r=60, t=60, b=100),
    hovermode='closest',
    font=dict(
        size=9,
        color='black',
        family='Arial'
    ),
)

fig_FR.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\FR_techno_mix_shares_production.png",
                   width=591, height=450, scale=3)

fig_FR.show()

# ====================================
# GRAPH 3: Germany Technology Mix
# ====================================

fig_DE = make_subplots(specs=[[{"secondary_y": True}]])

for tech in sorted_techs:
    if tech in df_yearly_pct_DE.columns:
        percentages = df_yearly_pct_DE[tech].values
        avg_pct = percentages.mean()

        hover_text = [f'{tech}<br>Share: {pct:.1f}%' for pct in percentages]

        fig_DE.add_trace(
            go.Bar(
                x=year_labels,
                y=percentages,
                name=tech,
                marker=dict(
                    color=tech_colors_plotly.get(tech, 'rgb(128, 128, 128)'),
                    line=dict(color='white', width=0.5)
                ),
                hovertemplate='%{hovertext}<extra></extra>',
                hovertext=hover_text,
                text=[f'{pct:.0f}%' if pct > 5 else '' for pct in percentages],
                textposition='inside',
                textfont=dict(size=8, color='white'),
                showlegend=bool(avg_pct >= 1.0)
            )
        )

# fig_DE.add_trace(
#     go.Scatter(
#         x=year_labels,
#         y=yearly_total_impact_DE.values,
#         name='Carbon intensity',
#         mode='lines',
#         line=dict(color='black', width=1, dash='dash'),
#         hovertemplate='DE carbon intensity: %{y:.2f} gCO2eq/kWh<extra></extra>'
#     ),
#     secondary_y=True
# )

fig_DE.update_xaxes(
    title_text='Month',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8
)

fig_DE.update_yaxes(
    title_text='German exportation mix to CH (%)',
    title_font=dict(size=12, color='black', family='Arial'),
    tickfont=dict(size=10, color='black', family='Arial'),
    showgrid=True,
    gridcolor='lightgray',
    gridwidth=0.8,
    range=[0, 100]
)

# fig_DE.update_yaxes(
#     title_text='Carbon intensity (gCO2eq/kWh)',
#     title_font=dict(size=12, color='black', family='Arial'),
#     tickfont=dict(size=10, color='black', family='Arial'),
#     showgrid=False,
#     secondary_y=True
# )

fig_DE.update_layout(
    template='plotly_white',
    barmode='stack',
    title=dict(
        text='Germany - Yearly technology mix for exportation',
        x=0.5,
        font=dict(size=14, color='black', family='Arial')
    ),
    legend=dict(
        font=dict(size=9, color='black', family='Arial'),
        orientation='h',
        xanchor='center',
        yanchor='top',
        x=0.5,
        y=-0.20,
        bgcolor='rgba(255,255,255,0)'
    ),
    margin=dict(l=60, r=60, t=60, b=100),
    hovermode='closest',
    font=dict(
        size=9,
        color='black',
        family='Arial'
    )
)

fig_DE.write_image(r"C:\Users\PC Jules\Desktop\Update 2024\Graph\DE_techno_mix_shares_production.png",
                   width=591, height=450, scale=3)

fig_DE.show()

#=======================================
# SUMMARY STATISTICS - COMPARISON
#=======================================

print("=" * 100)
print("FRANCE vs GERMANY EXPORTATION for CH 2024 - COMPARATIVE ANALYSIS")
print("=" * 100)

# --- 1. ANNUAL PRODUCTION COMPARISON ---
print("\n--- ANNUAL EXPORTATION for CH OVERVIEW ---")
print(f"{'Metric':<40} {'France':<25} {'Germany':<25}")
print("-" * 100)

annual_total_FR = yearly_total_prod_FR.sum()
annual_total_DE = yearly_total_prod_DE.sum()
print(f"{'Total Annual Exportation for CH (GWh)':<40} {annual_total_FR:>20,.2f}   {annual_total_DE:>20,.2f}")

yearly_avg_FR = yearly_total_prod_FR.mean()
yearly_avg_DE = yearly_total_prod_DE.mean()
print(f"{'Average yearly Exportation for CH (GWh)':<40} {yearly_avg_FR:>20,.2f}   {yearly_avg_DE:>20,.2f}")

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
max_month_FR = month_names[yearly_total_prod_FR.idxmax().month - 1]
max_month_DE = month_names[yearly_total_prod_DE.idxmax().month - 1]
print(f"{'Highest Exportation for CH Month':<40} {max_month_FR:>20}   {max_month_DE:>20}")

# --- 2. CARBON INTENSITY COMPARISON ---
print("\n--- CARBON INTENSITY ANALYSIS ---")
print(f"{'Metric':<40} {'France':<25} {'Germany':<25}")
print("-" * 100)

annual_avg_impact_FR = yearly_total_impact_FR.mean()
annual_avg_impact_DE = yearly_total_impact_DE.mean()
print(f"{'Annual Average (gCO2eq/kWh)':<40} {annual_avg_impact_FR:>20.2f}   {annual_avg_impact_DE:>20.2f}")

cleanest_month_FR = month_names[yearly_total_impact_FR.idxmin().month - 1]
cleanest_month_DE = month_names[yearly_total_impact_DE.idxmin().month - 1]
print(f"{'Cleanest Month':<40} {cleanest_month_FR:>20}   {cleanest_month_DE:>20}")

dirtiest_month_FR = month_names[yearly_total_impact_FR.idxmax().month - 1]
dirtiest_month_DE = month_names[yearly_total_impact_DE.idxmax().month - 1]
print(f"{'Dirtiest Month':<40} {dirtiest_month_FR:>20}   {dirtiest_month_DE:>20}")

# --- 3. TOP TECHNOLOGIES ---
print("\n--- TOP 5 TECHNOLOGIES BY ANNUAL EXPORTATION FOR CH ---")
print("\nFRANCE:")
annual_prod_by_tech_FR = df_prod_m_FR.sum()
sorted_by_prod_FR = annual_prod_by_tech_FR.sort_values(ascending=False)
for i, (tech, prod) in enumerate(sorted_by_prod_FR.head(5).items(), 1):
    pct = (prod / annual_total_FR) * 100
    print(f"  {i}. {tech}: {prod:,.2f} GWh ({pct:.1f}%)")

print("\nGERMANY:")
annual_prod_by_tech_DE = df_prod_m_DE.sum()
sorted_by_prod_DE = annual_prod_by_tech_DE.sort_values(ascending=False)
for i, (tech, prod) in enumerate(sorted_by_prod_DE.head(5).items(), 1):
    pct = (prod / annual_total_DE) * 100
    print(f"  {i}. {tech}: {prod:,.2f} GWh ({pct:.1f}%)")

# --- 4. RENEWABLE vs NON-RENEWABLE ---
print("\n--- RENEWABLE vs NON-RENEWABLE ENERGY ---")

renewable_keywords = ['Hydro', 'Solar', 'Wind', 'Biomass', 'Geothermal', 'Waste']

renewable_techs_FR = [col for col in df_prod_m_FR.columns if any(x in col for x in renewable_keywords)]
non_renewable_techs_FR = [col for col in df_prod_m_FR.columns if col not in renewable_techs_FR]
renewable_prod_FR = df_prod_m_FR[renewable_techs_FR].sum().sum()
non_renewable_prod_FR = df_prod_m_FR[non_renewable_techs_FR].sum().sum()

renewable_techs_DE = [col for col in df_prod_m_DE.columns if any(x in col for x in renewable_keywords)]
non_renewable_techs_DE = [col for col in df_prod_m_DE.columns if col not in renewable_techs_DE]
renewable_prod_DE = df_prod_m_DE[renewable_techs_DE].sum().sum()
non_renewable_prod_DE = df_prod_m_DE[non_renewable_techs_DE].sum().sum()

print(f"\n{'Energy Type':<40} {'France':<25} {'Germany':<25}")
print("-" * 100)
print(f"{'Renewable Energy (GWh)':<40} {renewable_prod_FR:>20,.2f}   {renewable_prod_DE:>20,.2f}")
print(f"{'Renewable Energy (%)':<40} {renewable_prod_FR/annual_total_FR*100:>20.1f}   {renewable_prod_DE/annual_total_DE*100:>20.1f}")
print(f"{'Non-Renewable Energy (GWh)':<40} {non_renewable_prod_FR:>20,.2f}   {non_renewable_prod_DE:>20,.2f}")
print(f"{'Non-Renewable Energy (%)':<40} {non_renewable_prod_FR/annual_total_FR*100:>20.1f}   {non_renewable_prod_DE/annual_total_DE*100:>20.1f}")

print("=" * 100)

FRANCE vs GERMANY EXPORTATION for CH 2024 - COMPARATIVE ANALYSIS

--- ANNUAL EXPORTATION for CH OVERVIEW ---
Metric                                   France                    Germany                  
----------------------------------------------------------------------------------------------------
Total Annual Exportation for CH (GWh)               44,071.51              65,124.60
Average yearly Exportation for CH (GWh)              6,295.93               9,303.51
Highest Exportation for CH Month                          Dec                    Dec

--- CARBON INTENSITY ANALYSIS ---
Metric                                   France                    Germany                  
----------------------------------------------------------------------------------------------------
Annual Average (gCO2eq/kWh)                              5.08                  64.86
Cleanest Month                                            Dec                    Dec
Dirtiest Month                             